In [14]:
import pandas as pd
from openai import AzureOpenAI
from data_process_public import extract_analysis,process_text,extract_json

client = AzureOpenAI(
    azure_endpoint="https://zonekey-gpt4o.openai.azure.com/",
    api_key="b01e3eb073fe43629982b30b3548c36e",
    api_version="2024-02-01"
)
prompt = """
后面的“待分析文本列表”是一组老师在课堂中说的话，请你对列表中的每个老师话语进行单独分析，预测在老师说完每段话后，学生要开展的课堂活动。课堂活动类别包括：个体发言、独立练习、学生齐写、小组学习、学生听写、学生齐读、多展示、集体未知。课堂活动的预测过程为：
step1：严格参照”示例“的分析过程，识别与课堂活动相关的特征短语或特征短句，先判定老师说完这段话后，学生要开展的课堂活动类别，再判定该课堂活动的进行状态，分为三个状态：开始、进行中、结束。输出分析过程，输出前缀为“分析过程：”，不需要重复输出老师的话语。
step2：输出一个结构化结果，为一个 JSON 对象，该对象包含3个属性，json对象的定义如下：
{
  "label": "string", 
  "status": "string", 
  "key_text": "string" 
}
其中，“label”填写的是step1中识别的课堂活动类别，如果识别多个课堂活动类别，”label"中填写多个课堂活动，中间用“、”来连接，”label"中不要重复填写课堂活动；"status"中填写的是step1中识别的课堂活动的进行状态；"key_text"中填写的是在step1中识别的与所预测的课堂活动相关的子句。如果step1中识别了多个课堂活动，"key_text"中填写与第一个课堂活动相关的子句。如果从老师的话语中无法识别出任何预设的课堂活动类别，则"label"、"status"和"key_text"字段都填写"NA"。 
请对列表中的每个老师话语单独处理，并输出相应的结果。
示例：
老师话语：“张华，请你上来，向大家展示你的科学项目，并解释你的发现。”
分析过程: 在这段话语中，”请你上来“体现出老师发起了一个开展课堂活动的指令。
首先“请你上来”体现出老师邀请一个学生到讲台或者教室前面的位置，进一步地，“向大家展示“体现出老师邀请学生向全班其他同学分享自己的科学项目。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生到讲台或者教室前面的位置向全班其他同学分享自己的科学项目，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：”橡皮是橡胶制成的，对，你来说“
分析过程：在这段话语中，“你来说”体现出老师发起了一个开展课堂活动的指令。
“你来说”中的”来说“体现出老师邀请学生参与一个分享活动，虽然从这段话中无法直接得知老师要求学生分享的具体内容是什么，但在课堂常规互动中，学生对老师的要求已经有预先的了解，尽管这一点在这段话中未被明确表示出来，但是，在老师的这段话之前已有相关的引导或说明。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生在全班其他同学面前分享自己的见解或成果，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“来，这段话中还有没有哪一个句子也能体现母鸡的负责？“
分析过程：在这段话语中，“这段话中还有没有哪一个句子也能体现母鸡的负责”这句话是老师询问学生课文中哪个句子体现母鸡的负责，老师提问这个问题，很明显希望学生能够分享自己的见解或答案。
综合上述整段话的分析过程，老师想要邀请学生在其他学生面前分享自己的见解，符合“个体发言”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“x分之10有没有问题，非常好，请坐正确。好，第二题，来这位同学后面这位。“
分析过程：在这段话语中，“来这位同学后面这位”体现出老师发起了一个开展课堂活动的指令。
首先，“好，第二题”体现出老师要求学生集中注意力在第二题上。接下来地，”这位同学后面那位“体现出老师邀请某位同学后面的那位同学参与一个课堂活动，结合上句”第二题“可知，老师邀请这位特定的学生参与的课堂活动是回答或解释第二小题。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生在其他学生面前分享自己的答案或见解，这个要素符合“个体发言”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：那同学们在我们学习前半部分的这个学习方法中，你有哪些收获呢？我们是通过什么样的方式方法来学习之前的这些部分呢？你能说一说吗？李新成。
分析过程：在这段话语中，“说一说“体现出老师发起了一个开展课堂活动的指令。
首先，“你能说一说吗”中的”说一说“体现出老师邀请学生参与一个分享活动，结合上一句”你有哪些收获“可知，老师邀请学生分享的是自己学习过程中的收获。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生在全班其他同学面前分享自己的收获，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“那么你说一下这些材料有什么特点面有什么功能呢？你想一想，布料的特点是什么？”
分析过程：在这段话语中，”那么你说一下”体现出老师发起了一个开展课堂活动的指令。
首先，”你说一下“体现出老师邀请学生在其他同学面前分享材料的特点和功能。接下来地，“你想一想，布料的特点是什么”这句话进一步体现了老师在提醒学生深入思考布料的特点，结合上一句”这些材料有什么特点面有什么功能“可知，老师先引导学生分享材料的特点和功能，之后又引导学生思考布料的特点，因为布料属于材料的一种，所以，这表明老师为了防止学生忘记分享的内容，再次提醒学生分享的重点可以放在布料的特点上。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生在其他学生面前分享自己的见解，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“好的，今天我们开始学习小蝌蚪找妈妈这篇新课文，小朋友们， 谁来读一读这篇课文的开头自然段？”
分析过程：在这段话语中，”谁来读一读这篇课文的开头自然段“体现出老师发起了一个开展一个课堂活动的指令。
“谁来读一读”体现出老师邀请一个或多个学生在其他学生面前朗读小蝌蚪找妈妈这篇课文的开头自然段，以此展示他们的朗读成果。
综合上述整段话的分析过程，这个课堂活动是老师邀请学生在其他学生面前分享自己的朗读成果，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：”接下来我们来做个角色扮演游戏。张三，你来扮演顾客，李四，你现来扮演店主，开始吧。“
分析过程：在这段话语中，”接下来我们来做个角色扮演游戏“体现出老师发起了一个开展课堂活动的指令。
首先。“来做个角色扮演游戏”明确了课堂活动的目的，即开展角色扮演活动。进一步地，“张三，你来扮演顾客，李四，你现来扮演店主。”中的”张三，你来“和”李四，你来“体现出老师挑选了两名学生分别扮演两个不同的角色，进而向其他学生分享自己对角色的理解。
综合上述整段话的分析过程，这个课堂活动是老师邀请两名学生在其他学生面前分享自己对角色的理解，符合”多人展示“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”。因为执行该课堂活动的指令刚刚发出，因此 进行状态为“开始”。
老师话语：”天黑前上床睡觉都是我们习以为常的。为什么在作者看来却需要慰藉呢？请同学们联系第四自然段想一想，好，你继续说。”
分析过程：在这段话语中，先分析第一个子文本段——”请同学们联系第四自然段想一想”，其中，”请同学们联系第四自然段想一想“体现出老师发起了一个开展课堂活动的指令。首先，”同学们“体现出该活动是由全体学生共同开展的，进一步地，”联系第四自然段想一想”体现出老师要求学生联系第四自然段进行思考活动，思考活动是个体的大脑活动，无法集体整齐划一地开展，只能由每一个学生独自完成。综合第一个子文本段的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主思考活动，符合“独立练习”的特征，因而，老师发出了一个要求学生开展“独立练习”的指令。
接下来，再分析第二个子文本段——“好，你继续说”，其中，”你继续说“体现出老师发起了开展第二个课堂活动的指令，其中”继续说”体现出老师邀请学生在其他同学面前分享之前没有说完的内容。综合第二个子文本段的分析过程，这个课堂活动是老师邀请学生在其他同学面前分享自己的学习成果，符合”个体发言“的特征，因而，第二个指令是开展”个体发言“的课堂活动的指令。
由于整段话语中没有使用明显的时间词汇来区分这两个指令的执行顺序，所以，无法确定老师说完这段话后具体执行哪个指令。因此，预测的课堂活动类别是“独立练习”或“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。
分析过程：在这段话语中，先分析第一个子文本段——“那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。”，其中，“请用坐伤的方式来完成”体现出老师发起了一个课堂活动的指令。首先“我们”一词体现出这个课堂活动是老师邀请全体学生共同参与的，”用坐伤的方式来完成这道题“体现出老师要求学生按照规定的方式来完成练习题，通常情况下，如果老师没有明确要求学生以小组讨论的方式完成，那么学生就需要独自完成练习题。综合第一个子文本段的分析过程，这个课堂活动需要全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，因而，老师发出了一个要求学生开展“独立练习”的指令。
接下来，再分析第二个子文本段——“那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。”，首先，”那我们现在小组分享一下”体现出老师指示小组成员进入下一个活动步骤——分享阶段。进一步地，“分享一下你的结果”体现出老师要求小组成员通过互相讨论的方式来展示各自的学习成果或者以小组的形式向全班展示学习成果，由于这两种情况都有可能，只能根据后文的内容再次进行判断。”看一下大家完成的是否是一致的“这里的”大家“可能指代每一个小组内的全部成员，也可能指代全班同学，所以还是无法明确判断”小组分享一下“是以哪种方式进行分享。综合第二个子文本段的分析过程，如果老师要求小组内部通过互相讨论的的方式来展示各自的学习成果，符合“小组学习”的特征，那么老师发出了一个要求学生开展“独立练习”的指令；如果老师要求学生以小组的形式展示各个小组的学习成果，则符合“多人展示”的特征，那么老师发出了一个要求学生开展“多人展示”的指令。
在整段话语中，“做商大家完成的非常快”体现出老师对学生完成任务的速度表示认可，这意味着第一个课堂活动——”独立练习“，已经结束，此外，“那我们现在小组分享一下”中的“现在”一词体现出老师要求立即执行第二个课堂活动——“小组学习”或”多人展示“。所以，老师说完这段话后，预测的课堂活动类别是“小组学习”或”多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“我希望每个小组现在讨论一下刚才的实验结果，等一会儿向全班汇报。”
分析过程：在这段话语中，先分析第一个子文本段——“我希望每个小组现在讨论一下刚才的实验结果“，其中，”每个小组现在讨论一下”体现出老师发起了一个开展课堂活动的指令，其中“讨论一下”体现出老师要求学生开展讨论活动，结合“每个小组”体现出这是小组内部的讨论活动。综合第一个子文本段的分析过程，这个课堂活动是老师要求小组内部通过互相讨论来达到自主学习的目的，符合“小组学习”的特征。因而，老师发出了一个要求学生开展“小组学习”的指令。
接下来，再分析第二个子文本段——“等一会儿向全班汇报”，其中”向全班汇报”体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生在其他同学面前做汇报工作，即要求学生分享汇报内容，由于第二个子文本段没有直接体现出该课堂活动的执行主体，只能结合第一个子文本段来看，“每个小组”暗示了分享活动的执行主体是每一个小组。综合第二个子文本段的分析过程，这个课堂活动是老师邀请每个小组在其他学生面前分享自己的汇报成果，符合”个体发言“的特征。因而，第二个指令是开展”个体发言“的课堂活动的指令。
在整段话语中，”等一会儿”这个词明显体现出第一个指令执行完成后，才执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前安排，不需要被立即执行，此外，“现在讨论一下”中的“现在”一词体现出老师要求立即执行第一个课堂活动——”小组学习“。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“请你们四人一组，探讨一下《红楼梦》中主要人物的性格特点。”
分析过程：在这段话语中，”请你们四人一组“体现出老师发起了一个开展课堂活动的指令。首先，“请你们四人一组”体现出老师要求学生以四人为一个小组开展课堂活动。进一步地，“探讨一下”体现出老师要求小组内部深入讨论《红楼梦》中主要人物的性格特点的活动。
综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“接下来，请大家自行解答第五页的练习题，不懂的地方可以标记出来。”
 分析过程：在这段话语中，”请大家自行解答第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
首先，“请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“自行解答第五页的练习题”中的”自行解答“体现出该课堂活动是由学生独自完成的。
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：大家开始做一下第五页的练习题，小明你上来做一下。
 分析过程：在这段话语中，“大家开始做一下第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
首先，“大家开始做一下”中的“大家”体现出这个课堂活动是由全体学生共同参与的，“做一下”体现出老师要求学生完成第五页的练习题，通常情况下，如果老师没有明确要求学生以小组讨论的方式完成，那么学生就需要独自完成练习题。进一步地，“小明你上来做一下”，其中“你上来”体现出老师要求学生到讲台上或班级前面的位置，“小明”可能是一个学生的名字，这里的”你“指代”小明“这位学生。”做一下“体现出老师要求该学生执行一个课堂活动，结合上一句话“第五页的练习题”可知，要求学生执行的课堂活动是完成第五页的练习题，然而，这句话并没有表明该学生需要展示或解释答案给其他同学听，只是上来完成练习题，这意味着该学生只是不和其他同学一样在自己的座位上完成练习题，而是换个位置完成。
综合上述整段话的分析过程，全体学生在同一时间内，不同位置各自独立完成练习题，这个课堂活动是由全体学生参与，且每个学生独立完成的自主练习活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“妈妈要到什么时候才要到夜里？那我饿了，饿了也得到太阳落下时才准吃。跟他一样。哎，你试着读一读，不用同学举手，自己读一读。“
分析过程：在这段话语中，“你试着读一读”体现出老师发起了一个开展课堂活动的指令。
首先，”你试着读一读“中的“你”可能指代一个学生，也可能指代全体学生。进一步地，“不用同学举手”中的“举手”是一种集体行为，如果班级只有一名同学，则不涉及“举手"这种行为，所以，“不用同学举手”中的“同学”指代全体同学，这也就意味着，上一句的”你试着读一读“中的”你”指代的也是全体同学。然后，“自己读一读”中的“自己”体现出老师要求学生独自朗读。
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“接下来我们进入课文，学习《春节的习俗》。请大家先看第一段，读一读，出声朗读，并反复诵读第三至第七自然段，边读边想象文字的场景。时间两分钟，开始。”
分析过程：在这段话语中，“请大家先看第一段”体现出老师发起了一个开展课堂活动的指令。
首先，”请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“边读边想象文字的场景”体现出老师要求学生一边朗读一边想象文字的场景，因为想象属于个体的大脑活动，无法集体整齐划一地开展，所以，只能由学生独自完成。 
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“好，同学们，刚才我们讨论的那个场景，作者并没有对它的具体细节进行描绘。同学们，你们能不能运用你的想象力，详细描述一下这场大火是如何吞噬了整个森林的？你想象一下。“
分析过程：在这段话语中，“运用你的想象力”体现出老师发起了一个课堂活动的指令。
首先，”同学们“体现出这个课堂活动是由全体学生共同参与的。接下来地，”运用你的想象力“和“描述一下”体现出老师要求学生利用想象力深入描述一个场景，由于想象活动是个体的大脑思维活动，无法集体整齐划一地开展，所以，只能由每一个学生独自完成。“你的想象力”中的“你”结合前面的“你们能不能”可知，这里的“你”指代的还是全体学生。进一步地，“你想象一下”老师再次提醒学生想象大火如何吞噬整个森林的情景，这里的“你”结合前面的“同学们”依然指代全体学生。因而，整个课堂活动都是由全体学生共同参与的。
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主想象活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“自学六七自然段默读自学提示，时间，七分钟，小组之间议一议，开始。”
分析过程：在这段话语中，先分析第一个子文本段——”自学六七自然段默读自学提示，时间，七分钟“，其中”自学六七自然段”体现出老师发起了开展一个课堂活动的指令，首先，“自学六七自然段”中的”自学“体现出老师要求学生独自按照自学提示来学习六七自然段。进一步地，”时间，七分钟“体现出这个自学活动要在规定的时间范围内完成，这样的时间规定通常情况下是为了方便老师对学生的统一管理，所以这个自学活动的执行主体是全体学生。综合第一个子文本段的分析过程，这个课堂活动由全体学生参与、且每一个学生独自完成的自学活动，符合“独立练习”的特征，因此，老师发出了一个要求学生开展”独立练习“课堂活动的指令。
接下来，再分析第二个子文本段——“小组之间议一议”，体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生以小组的形式开展内部讨论活动。综合第二个子文本段的分析过程，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，因而，第二个指令是开展”小组学习“的课堂活动的指令。
由于这段话中，“时间，七分钟“这个短语明显体现出第一个指令开展七分钟后，才开始执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前预告或提前安排，不需要被立即执行。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：”在干涸的河床上，一只乌鸦辛苦地寻找水源。它终于在一个石头缝中发现了一点水。来，让我们一起来感受一下乌鸦找水的艰辛。”
分析过程：在这段话语中，“让我们一起来感受一下乌鸦找水的艰辛。“体现出老师发起了一个开展课堂活动的指令。
首先，“让我们一起来”中的”我们“一词体现出这个课堂活动是老师邀请全体学生共同参与的，其中“一起”有两种可能的理解：第一种理解所有学生整齐划一地进行同一个活动；第二种理解是每位学生各自独立参与，但都在做同样的事情。进一步地，“感受一下”体现出老师邀请学生参与感受活动，因为感受活动是个体的心理体验活动，无法集体整齐划一地开展，只能由每一个学生独自完成，因而，这段话语中的”一起“是第二种理解。 
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主感受活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“请我们像他一样，用一句话，把这个故事哎就讲出来了，来，我们一起来试试看。”
分析过程：在这段话中，“我们一起来试试看”体现出老师发起了开展一个课堂活动的指令。
首先，“我们一起来试试看“中”的”我们“体现出这个课堂活动是老师邀请全体学生共同参与的。“一起”这个词有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的事情。进一步地，‘试试看“体现出老师想要学生尝试做一件事情，结合上文“用一句话，把这个故事哎就讲出来了“可知，老师想要学生尝试的是用一句话将故事复述出来，因为复述活动是个体经过大脑思考后才能得出结论的活动，这种思考活动很难集体整齐划一地开展，只能由每一个学生独自完成，因而，这里的“一起”是第二种理解 。综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主复述活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“现在我会念一段诗句，大家请认真听写下来。”
分析过程：在这段话语中，”请认真听写“体现出老师发起了开展一个课堂活动的指令。
首先，”大家“体现出这个课堂活动是由全体学生共同参与的。进一步地，”认真听写“中的”听写“一词很明显体现出老师让学生开展听写活动。
综合上述子文本段的分析过程，这个课堂活动是老师邀请全体学生参与的听写活动，符合”学生听写“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生听写”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“好，这课要请男孩子来做杜鹃，好不好？把这个话来读一读啊，正当桃树成活的时候，预备起，正当桃树成活的时候”
分析过程：在这段话语中，“这课要请男孩子来做杜鹃“体现出老师发起了一个开展课堂活动的指令。
首先，”这课要请男孩子来做杜鹃”中的”男孩子”，因为没有“一个”这类量词来修饰“男孩子“这个词，所以，这里的”男孩子“指代全体男同学这个群体。“来做杜鹃”体现了老师要求男同学这个群体扮演杜鹃这个角色，但并未明确阐述扮演杜鹃的具体目的或随后的活动内容。接下来地，“把这个话来读一读”中的“读一读”体现出老师让男同学扮演杜鹃的目的是通过这种角色扮演方式来朗诵杜鹃的台词，是一种出声朗读活动。然后，“预备起”这个词体现出这个出声朗读活动需要学生整齐划一地开展。
综合上述整段话语的分析过程，这个出声朗读活动是老师邀请全体男同学这个群体参与、且整齐划一地开展，符合“学生齐读”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“我来读前面的你们来读第二大段，我一向讨厌母鸡。”
分析过程：在这段话语中，”我来读前面的你们来读第二大段“体现出老师发起了一个开展课堂活动的指令。
首先，“我来读前面的，你们来读第二大段”明确分配了朗读任务，其中“我”指老师自身，而“你们”则代表在场的全体学生。这样的用词表明了一种明确的分工：老师首先领读文章的开头部分，随后由学生集体跟读后续部分。进一步地，这种老师领读，全体学生跟读的课堂活动，通常情况下，老师要求学生整齐划一地开展。
综合上述整段话的分析过程，这个课堂活动是老师领读，全体学生整齐划一地跟读，符合”学生齐读“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：”那让我们合作来读一读老师来读看到的部分，你们来读想象到的部分。最后一句话我们一起来读，好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”
分析过程：在这段话语中，“最后一句话我们一起来读“体现出老师发起了一个开展课堂活动的指令。
首先，“最后一句话我们一起来读”中的“我们一起来”体现出这个课堂活动是老师邀请学生集体参与的。“读”字体现出老师邀请学生参与出声朗读活动。由于后面的”好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”信息不明确，通过这句话只知道朗读的内容，无法进一步推测朗读活动的具体执行形方式：学生集体整齐划一地朗读还是每一位学生独自朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
综合上述整段话的分析过程，由于没有明确的信息来判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令，所以，老师说完这段话后，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：”同学们，大声的朗读课文。“
分析过程：在这段话语中，”大声的朗读课文“体现出老师发起了一个课堂活动的指令。
首先，”同学们“体现出老师邀请学生集体参与课堂活动。接下来地，“大声的朗读课文”明确了活动的具体内容，即学生们需要在朗读课文时保持较大的音量，这句话没有进一步明确朗读活动的具体执行形式：学生集体整齐划一地出声朗读还是每一位学生独自出声朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
综合上述整段话的分析过程，由于没有明确的信息判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令。因此，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“他跪在地上旁边呢有个人拿着拐杖在打他的小屁股，对不对？就是叫他知错就改对的啊，所以，这个改字我们一起来写一写，来，小手笔伸出来一横折，预备起，一横折，二横三十五，“
分析过程：在这段话语中，”这个改字我们一起来写一写“体现出老师发起了一个开展课堂活动的指令。
首先，“我们一起来写一写”中的”我们“体现出这个课堂活动是老师邀请学生集体参与的。“一起”有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的行为。”写一写“体现出老师要求学生开展书写活动。进一步地，"预备起"这个词体现出这个书写活动要求学生整齐划一地开展，因而，这里的”一起“是第一种理解。 
综合上述整段话的分析过程，这个书写活动是老师邀请学生集体参与、且整齐划一地开展，符合“学生齐写”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐写”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“让这个画面更怎么样了，做到了更有活力了。很好，你们都感受到了燕子加入了这幅图中，为这幅图增添了很多生趣。哎，现在我们开始闯第三关，“
分析过程：在这段话语中，“我们开始闯第三关”体现出老师发出了一个课堂活动的指令。
首先，“我们”这一词语明确体现出这个课堂活动是由全体学生集体参与的，而不是针对个别学生的指令。进一步地，“闯第三关”体现出老师要求学生参与一个闯关活动，根据这句话只能知道学生要参与一个课堂活动，但没有明确说明该课堂活动的内容细节以及参与形式。
综合上述整段话的分析过程，这个课堂活动是由全体学生共同参与、且活动内容和参与形式都不明确，学生需要等待进一步的指导和安排，符合”集体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“集体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“好，最后这位同学来尝试一下， ”
分析过程：在这段话语中，“最后这位同学来尝试一下”体现出老师发起了一个课堂活动的指令。
首先，“最后这位同学”体现出老师邀请一名特定的学生参与课堂活动。接下来地，“来尝试一下”体现出老师让该学生试着参与一个课堂活动，虽然这句话不能体现出老师要求学生尝试开展什么课堂活动，但是特定学生的”尝试“一般是尝试展示成果、回答问题或者操作课件，无论是哪一种课堂活动，都可视为一种展示行为。
综合上述整段话的分析过程，这个课堂活动是老师邀请特定学生在其他同学面前展示自己的某项技能，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。
老师话语：“好，你是通过圈一圈的方法来学习，请坐张鹤“
分析过程：在这段话语中，”你是通过圈一圈的方法来学习“这句话是老师对学生学习方法的概括，其中的”好“字是老师对学生学习方法的肯定。”请坐张鹤”中的”张鹤“可能是一位学生的名字，体现出学生已经回答完毕，老师请张鹤这位同学坐下。综合上述整段话分析过程，这段话只是在确认学生的回答，并没有明确的开展课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。
老师话语：“明天，我们有一个小组展示，每个小组需要准备一个关于环境保护的短片。”
分析过程：在这段话语中，“明天，我们有一个小组展示”体现出明天将会有一个小组展示的活动，虽然，这句胡体现出老师要求学生以小组的形式分享自己的成果，符合”多人展示“的特征，但是，“明天”一词明确指出了活动发生的时间是明天，而不是在今天这个课堂上，意味着这是老师对未来课堂活动的提前预告或安排，而不需要学生立即执行。
综合上述整段话的分析过程，这段话语的本质是布置了一项课后作业，需要学生们在明天的课堂上进行展示，而非一个需要立即执行的课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。
老师话语：”好，你们再想想我去听别人别的组啊。“
分析过程：在这段话语中，“我去听别人别的组”体现出老师要离开当前这个小组，接着去别的小组那里，这表明老师正在对每个小组进行逐一辅导，此外，这也表明每个小组成员正在积极参与小组讨论活动。其次，“你们再想想”中的“你们”指代当前小组的全体成员，而“再想想”则表示老师希望当前小组继续深入思考和讨论问题，这进一步支持了小组讨论的活动正在进行中的观点。
综合上述整段话的分析过程，班级被划分为多个小组，每个小组均在积极地进行讨论和交流。老师在完成对一个小组的指导后，便鼓励学生继续思考和讨论，随后前往下一个小组进行辅导。因此，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为每个小组正在积极地参与讨论，而老师则在各小组间巡视，因此进行状态为“进行中”。
老师话语：“有的同学读完了，已经开始在写任务单了。”
分析过程：在这段话语中，“有的同学读完了，已经开始在写任务单了“说明了一部分学生完成了阅读任务，并且已经在写任务单，这体现了班级内学生的行为已经出现分化，即不是所有学生都同时完成阅读或写作，从而说明“阅读”和“写任务单”两个任务被全体学生共同承担，但每一个学生完成这两个任务的时长各不相同。进一步地，由于每一个学生完成这两个任务的具体时长不同，所以，这表明每一个学生都在独自执行任务，没有其他同学的协助。
综合整段话的分析过程，尽管这段话没有体现出老师发起了一个开展课堂活动的指令，但是很明显老师在提醒学生课堂活动的进展状态，这意味着这是一个正在进行中的课堂活动，并且这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，所以，在老师说完这段话后，预测的课堂活动类别依旧是“独立练习”，因为这是一个正在进行中的课堂活
动，因此进行状态为“进行中”。

待分析文本列表：
"""

df = pd.read_excel("../../data/test_llm/高希娜.xlsx")
# 计算总长度
total_length = len(df)
# 计算 16 为一个单位的总数（取整数）
unit_length = 16
step = 2
unit_count = total_length // unit_length

# 定义 messages 列表
messages_list = []

# 循环处理每 16 个单位
for i in range(unit_count + 1):  # 包括最后不足16个的部分
    messages = [{"role": "system", "content": prompt}]
    for j in range(0, unit_length, step):
        start_idx = i * unit_length + j
        end_idx = start_idx + step
        if start_idx >= total_length:  # 如果起始索引超出总长度，跳出循环
            break
        if end_idx > total_length:  # 如果结束索引超出总长度，只取到总长度
            end_idx = total_length
        content = str(df['text'].to_list()[start_idx:end_idx])
        messages.append({"role": "user", "content": content})
    messages_list.append(messages)

# 打印每组 messages 的内容
for idx, messages in enumerate(messages_list):
    print(f"Message group {idx + 1}:")
    for message in messages:
        print(message)

Message group 1:
{'role': 'system', 'content': '\n后面的“待分析文本列表”是一组老师在课堂中说的话，请你对列表中的每个老师话语进行单独分析，预测在老师说完每段话后，学生要开展的课堂活动。课堂活动类别包括：个体发言、独立练习、学生齐写、小组学习、学生听写、学生齐读、多展示、集体未知。课堂活动的预测过程为：\nstep1：严格参照”示例“的分析过程，识别与课堂活动相关的特征短语或特征短句，先判定老师说完这段话后，学生要开展的课堂活动类别，再判定该课堂活动的进行状态，分为三个状态：开始、进行中、结束。输出分析过程，输出前缀为“分析过程：”，不需要重复输出老师的话语。\nstep2：输出一个结构化结果，为一个 JSON 对象，该对象包含3个属性，json对象的定义如下：\n{\n  "label": "string", \n  "status": "string", \n  "key_text": "string" \n}\n其中，“label”填写的是step1中识别的课堂活动类别，如果识别多个课堂活动类别，”label"中填写多个课堂活动，中间用“、”来连接，”label"中不要重复填写课堂活动；"status"中填写的是step1中识别的课堂活动的进行状态；"key_text"中填写的是在step1中识别的与所预测的课堂活动相关的子句。如果step1中识别了多个课堂活动，"key_text"中填写与第一个课堂活动相关的子句。如果从老师的话语中无法识别出任何预设的课堂活动类别，则"label"、"status"和"key_text"字段都填写"NA"。 \n请对列表中的每个老师话语单独处理，并输出相应的结果。\n示例：\n老师话语：“张华，请你上来，向大家展示你的科学项目，并解释你的发现。”\n分析过程: 在这段话语中，”请你上来“体现出老师发起了一个开展课堂活动的指令。\n首先“请你上来”体现出老师邀请一个学生到讲台或者教室前面的位置，进一步地，“向大家展示“体现出老师邀请学生向全班其他同学分享自己的科学项目。\n综合上述整段话的分析过程，这个课堂活动是老师邀请学生到讲台或者教室前面的位置向全班其他同学分享自己的科学项目，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂

In [25]:
import json
list_input = [item for item in messages if item.get("role") == "user"]
list_demo = [sub_item for item in list_input for sub_item in eval(item.get("content"))]
list_demo 

['刚刚我们用了三种不同的方式来解决了这个糖水更甜的问题。那现在朱老师有个问题，在这个过程当中，你觉得收获最大的或者体会最深的是什么呢，',
 '来哪位同学来谈谈，好？来第一位男同学，',
 '类比的思想是不是对很棒好，还有没有没有了好，来哪位同学来继续分享一下。好，最右边这位同学，',
 '还有没有补充好，来这位同学，',
 '我们可以产生分式。我们小学又学了在整数和分数，我们将它通称为有理数，从而使得我们的数域是不是扩大了？那同学们整式和分式我们可以通称为，',
 '它其实也可以帮助我们去解决更多生活当中的问题。那同学们，现在你可不可以帮周老师解决一下这样一个实际问题。算一算，周老师到底第二次加了多少克的糖尝试解决，',
 '好。同学们，那我看到很多同学至少都已经完成了列式，对不对？好，可能计算当中稍微有一点困难没有关系。来哪位同学分享一下你的做法来怎么做的，好不一定计算出了结果，对不对？你列出了式子都可以好来。最后这位男同学，',
 '你将这样的方程称为分式方程。那么这个分式方程我们今天大家想一想，我们能不能把它解出来，那我们思考一下它解决的方向。那你们觉得我们怎么样去解这个分式方程，来这位同学，',
 '交叉相乘。你交叉相乘的目的是想要把它变成一个什么整式方程哦，转化为整式方程，',
 '所以这这节课其实我想要构建的是我们本章内容的一个框架。那同学们，你能不能够回顾一下我们刚刚的这个经历的过程类比于分数的我们的学习谈一谈。你觉得我们分式这一章要学些什么内容，']

In [2]:
import re 
def extract_analysis(text):
    start_index = text.find("分析过程：")
    if start_index != -1:
        return text[start_index:]
    return ""

def process_text(text):
    # 查找关键词“json 复制代码”
    keyword1 = "json\n复制代码"
    # 如果找到关键词，则去除关键词及其后面的所有文本
    keyword2 = "```json"
    keyword3 = "{"
    if keyword1 in text:
        text = text.split(keyword1)[0]
    if keyword2 in text:
        text = text.split(keyword2)[0]
    if keyword3 in text:
        text = text.split(keyword3)[0]
    text = text.replace("\n", " ")
    return text

def extract_json(text):
    text = re.search(r'\{(?:[^{}]*|\{[^{}]*\})*\}', text).group(0)
    text = text.replace("\n", " ")
    return text


In [3]:
# 调用 API
list_output = []
for messages in messages_list:
    response = client.chat.completions.create(
        model="soikit_test",
        messages=messages,
    )
    content = response.choices[0].message.content

    break
    
# todo:后续的工作需要把跑出来的结果存储到combined.xlsx文件中。

In [19]:
response

ChatCompletion(id='chatcmpl-9ivYJwyYRXeRMptP5Nz4pm1z0UZBE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='老师话语：“我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，"\n分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n老师话语：“哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，"\n分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。\n综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题"\n}\n```\n\n老师话语：“好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，"\n分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老师点名“这位同学”，指明了开始展示的小组，并进一步引导。

In [4]:
content

'老师话语：“我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，"\n分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n老师话语：“哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，"\n分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。\n综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题"\n}\n```\n\n老师话语：“好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，"\n分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老师点名“这位同学”，指明了开始展示的小组，并进一步引导。\n综合上述整段话的分析过程，这个课堂活动是老师邀请某个小组在其他同学面前分享学习成果，符合“多人展示”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "多人展示",\n  "status":

In [6]:
content.split("老师话语：")

['',
 '“我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，"\n分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n',
 '“哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，"\n分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。\n综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题"\n}\n```\n\n',
 '“好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，"\n分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老师点名“这位同学”，指明了开始展示的小组，并进一步引导。\n综合上述整段话的分析过程，这个课堂活动是老师邀请某个小组在其他同学面前分享学习成果，符合“多人展示”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "多人展示",\n  "status"

In [11]:
list1 = [item for item in content.split("老师话语") if item !=""]
# 使用函数处理列表中的每个元素
analysis_processes = [extract_analysis(item) for item in list1]

list_output = [f'{process_text(item)}\n\n{extract_json(item)}'   for item in analysis_processes]
list_output

['分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。 \n\n{   "label": "NA",   "status": "NA",   "key_text": "NA" }',
 '分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。 综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。 \n\n{   "label": "小组学习",   "status": "开始",   "key_text": "先四人小组讨论一下借助学习卡上的问题" }',
 '分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老师点名“这位同学”，指明了开始展示的小组，并进一步引导。 综合上述整段话的分析过程，这个课堂活动是老师邀请某个小组在其他同学面前分享学习成果，符合“多人展示”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。 \n\n{   "label": "多人展示",   "status": "开始",   "key_text": "来拿个小组来分享一下自己的想法" }',
 '分析过程：“大家看他根据刚才的这个情境呢设出了水的质量之后，得到了两个代数式”这句话是老师对课堂内容的陈述，“这两个代数式是我们熟悉的整式吗”是一个开放性的问题，要求学生思考或提供答案。这些都没有明确学生要进行的活动。所以，无法识别出任何预设的课堂活动类别。 \n\n{   "label": "NA",   "status": "NA",   "k

In [12]:
list1 +list_output

['：“我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，"\n分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n',
 '：“哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，"\n分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。\n综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题"\n}\n```\n\n',
 '：“好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，"\n分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老师点名“这位同学”，指明了开始展示的小组，并进一步引导。\n综合上述整段话的分析过程，这个课堂活动是老师邀请某个小组在其他同学面前分享学习成果，符合“多人展示”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n```json\n{\n  "label": "多人展示",\n  "status": 

### qwen



In [48]:
import random
from http import HTTPStatus
import dashscope

dashscope.api_key = "sk-f089718a48534c4c84a0cfbc35e9fd1a"

df = pd.read_excel("../../data/test_llm/高希娜.xlsx")
prompt = """
    后面的“待分析文本”是一段老师在课堂中说的话，请你预测在老师说完这段话后，学生要开展的课堂活动。课堂活动类别包括：个体发言、独立练习、学生齐写、小组学习、学生听写、学生齐读，多人展示，集体未知，个体未知。课堂活动的预测过程为：
    step1：严格参照”示例“的分析过程，识别与课堂活动相关的特征短语或特征短句，先判定老师说完这段话后，学生要开展的课堂活动类别，再判定该课堂活动的进行状态，分为三个状态：开始、进行中、结束。输出分析过程，输出前缀为“分析过程：”，不需要重复输出老师的话语。
    step2：输出一个结构化结果，为一个 JSON 对象，输出前缀为”输出结果：“，该 JSON 对象包含3个属性，定义如下：
    {
      "label": "string", 
      "status": "string", 
      "key_text": "string" 
    }
    其中，“label”填写的是step1中识别的课堂活动类别，如果识别多个课堂活动类别，”label"中填写多个课堂活动，中间用“、”来连接，”label"中按照顺序填写识别出的课堂活动；"status"中填写的是step1中识别的课堂活动的进行状态；"key_text"中填写的是在step1中识别的与所预测的课堂活动相关的子句。如果step1中识别了多个课堂活动，"key_text"中填写与第一个课堂活动相关的子句。如果从老师的话语中无法识别出任何预设的课堂活动类别，则"label"、"status"和"key_text"字段都填写"NA"。


    示例：
    老师话语：“张华，请你上来，向大家展示你的科学项目，并解释你的发现。”
    分析过程: 在这段话语中，”张华，请你上来“体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动，”张华“可能是被邀请学生的名字。进一步地，“解释你的发现“体现出该学生被要求用话语来解释、分享自己的发现。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”橡皮是橡胶制成的，对，你来说“
    分析过程：在这段话语中，“你来说”体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动，“说”字体现出该学生被要求用话语来描述自己的见解。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“x分之10有没有问题，非常好，请坐正确。好，第二题，来这位同学后面这位。“
    分析过程：在这段话语中，“来这位同学后面这位”体现出老师发起了一个开展课堂活动的指令，邀请班级内特定位置的某个同学开展活动，结合上一句”第二题“可知，这位特定的学生被要求用话语来回答或解释第二小题。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：那同学们在我们学习前半部分的这个学习方法中，你有哪些收获呢？我们是通过什么样的方式方法来学习之前的这些部分呢？你能说一说吗？李新成。
    分析过程：在这段话语中，“你能说一说吗？李新成“体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动,”李新成“可能是被邀请学生的名字，结合上一句”你有哪些收获“可知，该学生被要求用话语来回答、分享自己的收获。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“那么你说一下这些材料有什么特点面有什么功能呢？你想一想，布料的特点是什么？”
    分析过程：在这段话语中，”那么你说一下”体现出老师发起了一个开展课堂活动的指令，邀请一位学生用话语来表达、分享材料的特点和功能。“你想一想，布料的特点是什么”这句话进一步体现了老师在提醒学生深入思考布料的特点，结合上一句”这些材料有什么特点面有什么功能“可知，老师先引导学生表达、分享材料的特点和功能，之后又引导学生思考布料的特点，此外，布料属于材料的一种，所以，这表明老师为了防止学生忘记表达、分享的内容，再次提醒学生表达的重点可以放在布料的特点上。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。


    老师话语：“好的，今天我们开始学习小蝌蚪找妈妈这篇新课文，小朋友们， 谁来读一读这篇课文的开头自然段？”
    分析过程：在这段话语中，”谁来读一读这篇课文的开头自然段“体现出老师发起了一个开展一个课堂活动的指令。首先，“小朋友们”体现出老师邀请全班同学共同参与课堂活动，“谁来读一读”中的“谁来”体现出老师会从全班同学中挑学生来朗读课文的开头自然段，如果老师接下来只挑选一个学生来朗读，即一位学生的话语行为，符合”个体发言“特征，老师发起了一个开展”个体发言“课堂活动的指令；如果老师接下来挑选多个同学分别开展朗读活动，即多位学生轮流分享各自的朗读成果，符合”多人展示“特征，则老师发起一个开展”多人展示“课堂活动的指令。综合上述整段话的分析过程，由于不知道老师说完这句话之后，会要求学生具体执行哪个课堂活动的指令，所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”或”多人展示“，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”接下来我们来做个角色扮演游戏。张三，你来扮演顾客，李四，你现来扮演店主，开始吧。“
    分析过程：在这段话语中，”接下来我们来做个角色扮演游戏“体现出老师发起了一个开展课堂活动的指令，这句话明确了课堂活动的目的是开展角色扮演活动。“张三，你来扮演顾客，李四，你现来扮演店主”进一步体现出老师挑选了两名学生来开展角色扮演活动，即通过角色扮演来分享他们对角色的理解。综合上述整段话的分析过程，这个课堂活动是由多个学生执行、且向全班同学分享他们对角色的理解，符合”多人展示“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”。因为执行该课堂活动的指令刚刚发出，因此 进行状态为“开始”。

    老师话语：”天黑前上床睡觉都是我们习以为常的。为什么在作者看来却需要慰藉呢？请同学们联系第四自然段想一想，好，你继续说。”
    分析过程：在这段话语中，先分析第一个子文本段——”请同学们联系第四自然段想一想”，这句话体现出老师发起了一个开展课堂活动的指令。首先，”同学们“体现出该活动是由全体学生共同开展的，进一步地，”联系第四自然段想一想”体现出老师要求学生联系第四自然段进行思考活动，思考活动是个体的大脑活动，无法集体整齐划一地开展，只能由每一个学生独自完成。综合第一个子文本段的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主思考活动，符合“独立练习”的特征，因而，老师发起了一个要求学生开展“独立练习”的指令。
    接下来，再分析第二个子文本段——“好，你继续说”，这句话体现出老师发起了开展第二个课堂活动的指令，该学生被要求用话语来表达自己没有说完的内容。综合第二个子文本段的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征，因而，第二个指令是开展”个体发言“的课堂活动的指令。
    由于整段话语中没有使用明显的时间词汇来区分这两个指令的执行顺序，所以，无法确定老师说完这段话后具体执行哪个指令。因此，预测的课堂活动类别是“独立练习”或“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。
    分析过程：在这段话语中，先分析第一个子文本段——“那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。”，其中，“请用坐伤的方式来完成”体现出老师发起了一个课堂活动的指令。首先“我们”一词体现出该课堂活动是全体学生共同参与的，”用坐伤的方式来完成这道题“体现出老师要求学生按照规定的方式来完成练习题，由于这是面向全体学生发出的指令，学生无法集体整齐划一的完成练习题，所以，只能每一个学生独自完成。综合第一个子文本段的分析过程，这个课堂活动需要全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，因而，老师发出了一个要求学生开展“独立练习”的指令。
    接下来，再分析第二个子文本段——“那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。”，其中，”那我们现在小组分享一下”体现出老师发起了开展一个课堂活动的指令，这里的“小组分享一下”表明学生需要在小组内部互相分享或以小组为单位向全班分享成果，由于这两种情况都有可能，只能根据后文的内容进行判断。”看一下大家完成的是否是一致的“这里的”大家“可能指代每一个小组内部的全部成员，也可能指代全班同学，所以还是无法明确判断”小组分享一下“是以哪种方式进行分享。综合第二个子文本段的分析过程，如果老师要求小组内部通过互相讨论的的方式来达到自主学习的目的，符合“小组学习”的特征，那么老师发出了一个要求学生开展“小组学习”的指令；如果老师要求学生以小组为单位向全班同学分享学习成果，则符合“多人展示”的特征，那么老师发出了一个要求学生开展“多人展示”的指令。
    在整段话语中，“做商大家完成的非常快”体现出老师对学生完成任务的速度表示认可，这意味着第一个课堂活动——”独立练习“，已经结束，此外，“那我们现在小组分享一下”中的“现在”一词体现出老师要求立即执行第二个课堂活动——“小组学习”或”多人展示“。所以，老师说完这段话后，预测的课堂活动类别是“小组学习”或”多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“我希望每个小组现在讨论一下刚才的实验结果，等一会儿向全班汇报。”
    分析过程：在这段话语中，先分析第一个子文本段——“我希望每个小组现在讨论一下刚才的实验结果“，其中，”每个小组现在讨论一下”体现出老师发起了一个开展课堂活动的指令，要求小组内部互相讨论实验结果。综合第一个子文本段的分析过程，这个课堂活动是老师要求小组内部通过互相讨论来达到自主学习的目的，符合“小组学习”的特征。因而，老师发出了一个要求学生开展“小组学习”的指令。
    接下来，再分析第二个子文本段——“等一会儿向全班汇报”，其中”向全班汇报”体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生在其他同学面前分享汇报内容，由于第二个子文本段没有直接体现出该课堂活动的执行主体，只能结合第一个子文本段来看，“每个小组”暗示了分享活动的执行主体是班级内每一个小组。综合第二个子文本段的分析过程，这个课堂活动是学生以小组为单位分享汇报成果，符合”多人展示“的特征。因而，第二个指令是开展”多人展示“的课堂活动的指令。
    在整段话语中，”等一会儿”这个词明显体现出第一个指令执行完成后，才执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前安排，不需要被立即执行，此外，“现在讨论一下”中的“现在”一词体现出老师要求立即执行第一个课堂活动——”小组学习“。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“请你们四人一组，探讨一下《红楼梦》中主要人物的性格特点。”
    分析过程：在这段话语中，”请你们四人一组“体现出老师发起了一个开展课堂活动的指令。首先，“请你们四人一组”体现出老师要求学生以四人为一个小组开展课堂活动。进一步地，“探讨一下”体现出老师要求小组内部深入讨论《红楼梦》中主要人物的性格特点的活动。
    综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“接下来，请大家自行解答第五页的练习题，不懂的地方可以标记出来。”
     分析过程：在这段话语中，”请大家自行解答第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
    首先，“请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“自行解答第五页的练习题”中的”自行解答“体现出该课堂活动是由每一个学生独自完成的。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：大家开始做一下第五页的练习题，小明你上来做一下。
     分析过程：在这段话语中，“大家开始做一下第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
    首先，“大家开始做一下”中的“大家”体现出这个课堂活动是由全体学生共同参与的，“做一下”体现出老师要求学生完成第五页的练习题，由于这是面向全体学生发出的指令，学生无法集体整齐划一的完成练习题，所以，只能每一个学生独自完成。进一步地，“小明你上来做一下”，其中“你上来”体现出老师要求学生到讲台上或班级前面的位置，“小明”可能是一个学生的名字，这里的”你“指代”小明“这位学生。”做一下“体现出老师要求该学生执行一个课堂活动，结合上一句话“第五页的练习题”可知，要求学生执行的课堂活动是完成第五页的练习题，然而，这句话并没有表明该学生需要展示或解释答案给其他同学听，只是上来完成练习题，这意味着该学生只是不和其他同学一样在自己的座位上完成练习题，而是换个位置完成。
    综合上述整段话的分析过程，全体学生在同一时间内，不同位置各自独立完成练习题，这个课堂活动是由全体学生参与，且每个学生独立完成的自主练习活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“妈妈要到什么时候才要到夜里？那我饿了，饿了也得到太阳落下时才准吃。跟他一样。哎，你试着读一读，不用同学举手，自己读一读。“
    分析过程：在这段话语中，“你试着读一读”体现出老师发起了一个开展课堂活动的指令。
    首先，”你试着读一读“中的“你”可能指代一个学生，也可能指代全体学生。进一步地，“不用同学举手”中的“举手”是一种集体行为，如果班级只有一名同学，则不涉及“举手"这种行为，所以，“不用同学举手”中的“同学”指代全体同学，这也就意味着，上一句的”你试着读一读“中的”你”指代的也是全体同学。然后，“自己读一读”中的“自己”体现出老师要求每一个学生独自朗读。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“接下来我们进入课文，学习《春节的习俗》。请大家先看第一段，读一读，出声朗读，并反复诵读第三至第七自然段，边读边想象文字的场景。时间两分钟，开始。”
    分析过程：在这段话语中，“请大家先看第一段”体现出老师发起了一个开展课堂活动的指令。
    首先，”请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“边读边想象文字的场景”体现出老师要求学生一边朗读一边想象文字的场景，因为想象属于个体的大脑活动，无法集体整齐划一地开展，所以，只能由学生独自完成。 
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，同学们，刚才我们讨论的那个场景，作者并没有对它的具体细节进行描绘。同学们，你们能不能运用你的想象力，详细描述一下这场大火是如何吞噬了整个森林的？你想象一下。“
    分析过程：在这段话语中，“运用你的想象力”体现出老师发起了一个课堂活动的指令。
    首先，”同学们“体现出这个课堂活动是由全体学生共同参与的。接下来地，”运用你的想象力“和“描述一下”体现出老师要求学生利用想象力深入描述一个场景，由于想象活动是个体的大脑思维活动，无法集体整齐划一地开展，所以，只能由每一个学生独自完成。“你的想象力”中的“你”结合前面的“你们能不能”可知，这里的“你”指代的还是全体学生。进一步地，“你想象一下”老师再次提醒学生想象大火如何吞噬整个森林的情景，这里的“你”结合前面的“同学们”依然指代全体学生。因而，整个课堂活动都是由全体学生共同参与的。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主想象活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“自学六七自然段默读自学提示，时间，七分钟，小组之间议一议，开始。”
    分析过程：在这段话语中，先分析第一个子文本段——”自学六七自然段默读自学提示，时间，七分钟“，其中”自学六七自然段”体现出老师发起了开展一个课堂活动的指令，首先，“自学六七自然段”中的”自学“体现出老师要求学生独自按照自学提示来学习六七自然段。进一步地，”时间，七分钟“体现出这个自学活动要在规定的时间范围内完成，这样的时间规定通常情况下是为了方便老师对学生的统一管理，所以这个自学活动的执行主体是全体学生。综合第一个子文本段的分析过程，这个课堂活动由全体学生参与、且每一个学生独自完成的自学活动，符合“独立练习”的特征，因此，老师发出了一个要求学生开展”独立练习“课堂活动的指令。
    接下来，再分析第二个子文本段——“小组之间议一议”，体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生以小组的形式开展内部讨论活动。综合第二个子文本段的分析过程，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，因而，第二个指令是开展”小组学习“的课堂活动的指令。
    由于这段话中，“时间，七分钟“这个短语明显体现出第一个指令开展七分钟后，才开始执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前预告或提前安排，不需要被立即执行。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”在干涸的河床上，一只乌鸦辛苦地寻找水源。它终于在一个石头缝中发现了一点水。来，让我们一起来感受一下乌鸦找水的艰辛。”
    分析过程：在这段话语中，“让我们一起来感受一下乌鸦找水的艰辛。“体现出老师发起了一个开展课堂活动的指令。
    首先，“让我们一起来”中的”我们“一词体现出这个课堂活动是老师邀请全体学生共同参与的，其中“一起”有两种可能的理解：第一种理解所有学生整齐划一地进行同一个活动；第二种理解是每位学生各自独立参与，但都在做同样的事情。进一步地，“感受一下”体现出老师邀请学生参与感受活动，因为感受活动是个体的心理体验活动，无法集体整齐划一地开展，只能由每一个学生独自完成，因而，这段话语中的”一起“是第二种理解。 
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主感受活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“请我们像他一样，用一句话，把这个故事哎就讲出来了，来，我们一起来试试看。”
    分析过程：在这段话中，“我们一起来试试看”体现出老师发起了开展一个课堂活动的指令。
    首先，“我们一起来试试看“中”的”我们“体现出这个课堂活动是老师邀请全体学生共同参与的。“一起”这个词有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的事情。进一步地，‘试试看“体现出老师想要学生尝试做一件事情，结合上文“用一句话，把这个故事哎就讲出来了“可知，老师想要学生尝试的是用一句话将故事复述出来，因为复述活动是个体经过大脑思考后才能得出结论的活动，这种思考活动很难集体整齐划一地开展，只能由每一个学生独自完成，因而，这里的“一起”是第二种理解 。综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主复述活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“现在我会念一段诗句，大家请认真听写下来。”
    分析过程：在这段话语中，”请认真听写“体现出老师发起了开展一个课堂活动的指令。
    首先，”大家“体现出这个课堂活动是由全体学生共同参与的。进一步地，”认真听写“中的”听写“一词很明显体现出老师让学生开展听写活动。
    综合上述子文本段的分析过程，这个课堂活动是老师邀请全体学生参与的听写活动，符合”学生听写“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生听写”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，这课要请男孩子来做杜鹃，好不好？把这个话来读一读啊，正当桃树成活的时候，预备起，正当桃树成活的时候”
    分析过程：在这段话语中，“这课要请男孩子来做杜鹃“体现出老师发起了一个开展课堂活动的指令。
    首先，”这课要请男孩子来做杜鹃”中的”男孩子”，因为没有“一个”这类量词来修饰“男孩子“这个词，所以，这里的”男孩子“指代全体男同学这个群体。“来做杜鹃”体现了老师要求男同学这个群体扮演杜鹃这个角色，但并未明确阐述扮演杜鹃的具体目的或随后的活动内容。接下来地，“把这个话来读一读”中的“读一读”体现出老师让男同学扮演杜鹃的目的是通过这种角色扮演方式来朗诵杜鹃的台词，是一种出声朗读活动。然后，“预备起”这个词体现出这个出声朗读活动需要学生整齐划一地开展。
    综合上述整段话语的分析过程，这个出声朗读活动是老师邀请全体男同学这个群体参与、且整齐划一地开展，符合“学生齐读”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“我来读前面的你们来读第二大段，我一向讨厌母鸡。”
    分析过程：在这段话语中，”我来读前面的你们来读第二大段“体现出老师发起了一个开展课堂活动的指令。
    首先，“我来读前面的，你们来读第二大段”明确分配了朗读任务，其中“我”指老师自身，而“你们”则代表在场的全体学生。这样的用词表明了一种明确的分工：老师首先领读文章的开头部分，随后由学生集体跟读后续部分。进一步地，这种老师领读，全体学生跟读的课堂活动，通常情况下，老师要求学生整齐划一地开展。
    综合上述整段话的分析过程，这个课堂活动是老师领读，全体学生整齐划一地跟读，符合”学生齐读“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”那让我们合作来读一读老师来读看到的部分，你们来读想象到的部分。最后一句话我们一起来读，好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”
    分析过程：在这段话语中，“最后一句话我们一起来读“体现出老师发起了一个开展课堂活动的指令。
    首先，“最后一句话我们一起来读”中的“我们一起来”体现出这个课堂活动是老师邀请学生集体参与的。“读”字体现出老师邀请学生参与出声朗读活动。由于后面的”好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”信息不明确，通过这句话只知道朗读的内容，无法进一步推测朗读活动的具体执行形方式：学生集体整齐划一地朗读还是每一位学生独自朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
    综合上述整段话的分析过程，由于没有明确的信息来判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令，所以，老师说完这段话后，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”同学们，大声的朗读课文。“
    分析过程：在这段话语中，”大声的朗读课文“体现出老师发起了一个课堂活动的指令。
    首先，”同学们“体现出老师邀请学生集体参与课堂活动。接下来地，“大声的朗读课文”明确了活动的具体内容，即学生们需要在朗读课文时保持较大的音量，这句话没有进一步明确朗读活动的具体执行形式：学生集体整齐划一地出声朗读还是每一位学生独自出声朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
    综合上述整段话的分析过程，由于没有明确的信息判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令。因此，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“他跪在地上旁边呢有个人拿着拐杖在打他的小屁股，对不对？就是叫他知错就改对的啊，所以，这个改字我们一起来写一写，来，小手笔伸出来一横折，预备起，一横折，二横三十五，“
    分析过程：在这段话语中，”这个改字我们一起来写一写“体现出老师发起了一个开展课堂活动的指令。
    首先，“我们一起来写一写”中的”我们“体现出这个课堂活动是老师邀请学生集体参与的。“一起”有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的行为。”写一写“体现出老师要求学生开展书写活动。进一步地，"预备起"这个词体现出这个书写活动要求学生整齐划一地开展，因而，这里的”一起“是第一种理解。 
    综合上述整段话的分析过程，这个书写活动是老师邀请学生集体参与、且整齐划一地开展，符合“学生齐写”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐写”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“让这个画面更怎么样了，做到了更有活力了。很好，你们都感受到了燕子加入了这幅图中，为这幅图增添了很多生趣。哎，现在我们开始闯第三关，“
    分析过程：在这段话语中，“我们开始闯第三关”体现出老师发出了一个课堂活动的指令。
    首先，“我们”这一词语明确体现出这个课堂活动是由全体学生集体参与的，而不是针对个别学生的指令。进一步地，“闯第三关”体现出老师要求学生参与一个闯关活动，根据这句话只能知道学生要参与一个课堂活动，但没有明确说明该课堂活动的内容细节以及参与形式。
    综合上述整段话的分析过程，这个课堂活动是由全体学生共同参与、且活动内容和参与形式都不明确，学生需要等待进一步的指导和安排，符合”集体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“集体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：课后作业做了没有？在二十九页，翻到二十九页。如果你是用网课的资料做的，你就把网课打印的资料拿出来。如果没有的话，你翻到课本二十九页。
    分析过程：在这段话语中，”把网课打印的资料拿出来”体现出老师发起了一个开展课堂活动的指令。
    首先，“如果你是用网课的资料做的“体现出班级内有一部分同学是用网课资料完成的课后作业，紧接着，“你就把网课打印的资料拿出来”中的”你“指代使用网课资料的这部分同学，“拿出来”只能体现出老师让这部分学生把资料拿出来，但老师未明确说明拿出资料后的具体操作。进一步地，”如果没有的话”结合上一句”如果你是用网课的资料“，体现出班级内有一部分同学，没有使用网课资料完成课后作业，紧接着，“你翻到课本二十九页”中的“你”指代没有使用网课资料的学生群体，“翻到”体现出没有使用网课资料的这部分同学翻书到二十九页，可以推测出这部分学生的课后作业写在了二十九页，同样，这里也未说明翻阅后需进行的具体活动。
    综合上述整段话语的分析过程，通过预设两种情况，老师确保了每个学生，无论他们以什么方式完成课后作业，都被纳入到这次课堂活动中，因而，说明这个课堂活动是由全体学生共同参与、且活动内容细节和参与形式都不明确，学生需要等待进一步的指导和安排，符合”集体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“集体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，最后这位同学来尝试一下， ”
    分析过程：在这段话语中，“最后这位同学来尝试一下”体现出老师发起了一个课堂活动的指令。
    首先，“最后这位同学”体现出老师邀请一名特定的学生参与课堂活动。接下来地，“来尝试一下”体现出老师让该学生试着做某件事情，但没有明确说明具体要做什么以及执行的具体形式也不清楚。
    综合上述整段话的分析过程，说明这是针对一名特定学生的指令，且指令内容和执行形式都不明确，学生需要等待进一步的指导和安排，符合”个体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，你是通过圈一圈的方法来学习，请坐张鹤“
    分析过程：在这段话语中，”你是通过圈一圈的方法来学习“这句话是老师对学生学习方法的概括，其中的”好“字是老师对学生学习方法的肯定。”请坐张鹤”中的”张鹤“可能是一位学生的名字，体现出学生已经回答完毕，老师请张鹤这位同学坐下。综合上述整段话分析过程，这段话只是在确认学生的回答，并没有明确的开展课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    老师话语：“明天，我们有一个小组展示，每个小组需要准备一个关于环境保护的短片。”
    分析过程：在这段话语中，“明天，我们有一个小组展示”体现出明天将会有一个小组展示的活动，虽然，这句胡体现出老师要求学生以小组的形式分享自己的成果，符合”多人展示“的特征，但是，“明天”一词明确指出了活动发生的时间是明天，而不是在今天这个课堂上，意味着这是老师对未来课堂活动的提前预告或安排，而不需要学生立即执行。
    综合上述整段话的分析过程，这段话语的本质是布置了一项课后作业，需要学生们在明天的课堂上进行展示，而非一个需要立即执行的课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    老师话语：”好，你们再想想我去听别人别的组啊。“
    分析过程：在这段话语中，“我去听别人别的组”体现出老师要离开当前这个小组，接着去别的小组那里，这表明老师正在对每个小组进行逐一辅导，此外，这也表明每个小组成员正在积极参与小组讨论活动。其次，“你们再想想”中的“你们”指代当前小组的全体成员，而“再想想”则表示老师希望当前小组继续深入思考和讨论问题，这进一步支持了小组讨论的活动正在进行中的观点。
    综合上述整段话的分析过程，班级被划分为多个小组，每个小组均在积极地进行讨论和交流。老师在完成对一个小组的指导后，便鼓励学生继续思考和讨论，随后前往下一个小组进行辅导。因此，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为每个小组正在积极地参与讨论，而老师则在各小组间巡视，因此进行状态为“进行中”。

    老师话语：“有的同学读完了，已经开始在写任务单了。”
    分析过程：在这段话语中，“有的同学读完了，已经开始在写任务单了“说明了一部分学生完成了阅读任务，并且已经在写任务单，这体现了班级内学生的行为已经出现分化，即不是所有学生都同时完成阅读或写作，从而说明“阅读”和“写任务单”两个任务被全体学生共同承担，但每一个学生完成这两个任务的时长各不相同。进一步地，由于每一个学生完成这两个任务的具体时长不同，所以，这表明每一个学生都在独自执行任务，没有其他同学的协助。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为这段话老师在提醒学生课堂活动的进展状态，意味着这是一个正在进行中的课堂活动，因此进行状态为“进行中”。

    老师话语：”所以它把水的质量设为了a克，非常好，来，我们先请坐来同学们，大家看他根据刚才的这个情境呢设出了水的质量之后，得到了两个代数式。这两个代数式是我们熟悉的整式吗，”
    分析过程：在这段话中，"我们先请坐来同学们，大家看他根据刚才的这个情境呢设出了水的质量之后，得到了两个代数式" 显示出该学生已完成回答，并被老师指引坐下。老师使用 "大家来看" 的表达方式来引导全班同学观察和思考刚才那位学生得到的两个代数式，接下来的询问 "这两个代数式是我们熟悉的整式吗" 旨在激发学生的思考和讨论，检测他们对代数式概念的理解。
    综合上述整段话的分析过程，这整段话语主要是老师的引导和询问，没有发起任何课堂活动。老师的语言主要是为了引导学生思考并对刚才的回答进行验证，而非开展新的教学活动。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    待分析文本如下：
    """

for i in range(len(df)):
    user_input=df.loc[i,'text']
    
    messages = [{'role': 'system', 'content': prompt},
                {'role': 'user', 'content': user_input}]
    
    
    response1 = dashscope.Generation.call(model="qwen-long",
                                         messages=messages,
                                         # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                         seed=random.randint(1, 10000),
                                         # 将输出设置为"message"格式
                                         result_format='message')
    if response1.status_code == HTTPStatus.OK:
        
        print("response",type(response1))
        
        print("response",response1["output"]['choices'][0]['message']['content']
    )
    df.loc[i,'qwen_long_output'] = response1["output"]['choices'][0]['message']['content']
   
df.to_excel("../../data/test_llm/高希娜.xlsx")


response <class 'dashscope.api_entities.dashscope_response.GenerationResponse'>
response 分析过程：在这段话语中，“那同学们”体现出老师可能要发起一个课堂活动的指令，但具体的活动内容尚未明确。老师提到了关于糖溶解在水中的现象，但没有明确指出学生需要做什么，比如发言、讨论或者实验操作等。

综合上述整段话的分析过程，这段话虽然包含可能引发课堂活动的引导，但没有提供足够的信息来识别出具体的课堂活动类别，因此，无法确定学生要开展的课堂活动。

输出结果：{"label": "集体未知", "status": "开始", "key_text": "那同学们"}


In [47]:
response1["output"]['choices'][0]['message']['content']


'材料：\n鸡蛋2个，西红柿1个，葱5克，油2汤匙（30ml)，盐1/4茶匙（1克），糖1/2茶匙（3克）\n\n做法：\n\n1. 鸡蛋打散后，加一点盐，一点水，用筷子顺时针搅拌均匀。\n2. 西红柿切成滚刀块，葱切末。\n3. 锅中放油，倒入葱末。\n4. 倒入鸡蛋液。\n5. 用铲子快速搅动，将鸡蛋搅成块状，盛出备用。\n6. 锅中再放少许油，倒入西红柿块。\n7. 放一点点糖，然后加入适量的盐。\n8. 翻炒至西红柿出红汤后，倒入炒好的鸡蛋块，翻炒均匀即可出锅。'

'ChatCompletion(id=\'chatcmpl-9ivYJwyYRXeRMptP5Nz4pm1z0UZBE\', choices=[Choice(finish_reason=\'stop\', index=0, logprobs=None, message=ChatCompletionMessage(content=\'老师话语：“我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，"\\n分析过程：这段话语并没有体现任何明确的课堂活动指令，而是老师在自述或讲解，可能是为后续的课堂活动做铺垫。因此，无法识别出任何预设的课堂活动类别。\\n```json\\n{\\n  "label": "NA",\\n  "status": "NA",\\n  "key_text": "NA"\\n}\\n```\\n\\n老师话语：“哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，"\\n分析过程：在这段话语中，“我们先四人小组讨论一下借助学习卡上的问题”体现出老师发起了一个开展课堂活动的指令。首先，“四人小组讨论一下”明确说明课堂活动的形式是小组内部的讨论，并强调每组有四名学生参与。进一步地，“分享你的想法”则是老师希望各小组成员之间相互交流和分享对学习卡上问题的见解。\\n综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。\\n```json\\n{\\n  "label": "小组学习",\\n  "status": "开始",\\n  "key_text": "先四人小组讨论一下借助学习卡上的问题"\\n}\\n```\\n\\n老师话语：“好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，"\\n分析过程：在这段话语中，“来拿个小组来分享一下自己的想法”体现出老师发起了一个开展课堂活动的指令。首先，“拿个小组来分享一下”明确说明课堂活动的形式是以小组的形式进行分享，进一步地，“大胆的分享”肯定并鼓励学生的积极性，最后，老

# doubao

In [59]:
import os
import pandas as pd
from tqdm import tqdm
import logging
from volcenginesdkarkruntime import Ark

# 配置日志
logging.basicConfig(filename='error.log', level=logging.ERROR)
# 设置API密钥为环境变量
os.environ["ARK_API_KEY"] = "f3c92d53-ebd7-4934-9cbe-322732cddb7e"

# 初始化Ark客户端
client = Ark(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
     region="cn-beijing"
)
prompt = """
    后面的“待分析文本”是一段老师在课堂中说的话，请你预测在老师说完这段话后，学生要开展的课堂活动。课堂活动类别包括：个体发言、独立练习、学生齐写、小组学习、学生听写、学生齐读，多人展示，集体未知，个体未知。课堂活动的预测过程为：
    step1：严格参照”示例“的分析过程，识别与课堂活动相关的特征短语或特征短句，先判定老师说完这段话后，学生要开展的课堂活动类别，再判定该课堂活动的进行状态，分为三个状态：开始、进行中、结束。输出分析过程，输出前缀为“分析过程：”，不需要重复输出老师的话语。
    step2：输出一个结构化结果，为一个 JSON 对象，输出前缀为”输出结果：“，该 JSON 对象包含3个属性，定义如下：
    {
      "label": "string", 
      "status": "string", 
      "key_text": "string" 
    }
    其中，“label”填写的是step1中识别的课堂活动类别，如果识别多个课堂活动类别，”label"中填写多个课堂活动，中间用“、”来连接，”label"中按照顺序填写识别出的课堂活动；"status"中填写的是step1中识别的课堂活动的进行状态；"key_text"中填写的是在step1中识别的与所预测的课堂活动相关的子句。如果step1中识别了多个课堂活动，"key_text"中填写与第一个课堂活动相关的子句。如果从老师的话语中无法识别出任何预设的课堂活动类别，则"label"、"status"和"key_text"字段都填写"NA"。


    示例：
    老师话语：“张华，请你上来，向大家展示你的科学项目，并解释你的发现。”
    分析过程: 在这段话语中，”张华，请你上来“体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动，”张华“可能是被邀请学生的名字。进一步地，“解释你的发现“体现出该学生被要求用话语来解释、分享自己的发现。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”橡皮是橡胶制成的，对，你来说“
    分析过程：在这段话语中，“你来说”体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动，“说”字体现出该学生被要求用话语来描述自己的见解。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“x分之10有没有问题，非常好，请坐正确。好，第二题，来这位同学后面这位。“
    分析过程：在这段话语中，“来这位同学后面这位”体现出老师发起了一个开展课堂活动的指令，邀请班级内特定位置的某个同学开展活动，结合上一句”第二题“可知，这位特定的学生被要求用话语来回答或解释第二小题。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：那同学们在我们学习前半部分的这个学习方法中，你有哪些收获呢？我们是通过什么样的方式方法来学习之前的这些部分呢？你能说一说吗？李新成。
    分析过程：在这段话语中，“你能说一说吗？李新成“体现出老师发起了一个开展课堂活动的指令，邀请一个学生开展活动,”李新成“可能是被邀请学生的名字，结合上一句”你有哪些收获“可知，该学生被要求用话语来回答、分享自己的收获。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“那么你说一下这些材料有什么特点面有什么功能呢？你想一想，布料的特点是什么？”
    分析过程：在这段话语中，”那么你说一下”体现出老师发起了一个开展课堂活动的指令，邀请一位学生用话语来表达、分享材料的特点和功能。“你想一想，布料的特点是什么”这句话进一步体现了老师在提醒学生深入思考布料的特点，结合上一句”这些材料有什么特点面有什么功能“可知，老师先引导学生表达、分享材料的特点和功能，之后又引导学生思考布料的特点，此外，布料属于材料的一种，所以，这表明老师为了防止学生忘记表达、分享的内容，再次提醒学生表达的重点可以放在布料的特点上。综合上述整段话的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。


    老师话语：“好的，今天我们开始学习小蝌蚪找妈妈这篇新课文，小朋友们， 谁来读一读这篇课文的开头自然段？”
    分析过程：在这段话语中，”谁来读一读这篇课文的开头自然段“体现出老师发起了一个开展一个课堂活动的指令。首先，“小朋友们”体现出老师邀请全班同学共同参与课堂活动，“谁来读一读”中的“谁来”体现出老师会从全班同学中挑学生来朗读课文的开头自然段，如果老师接下来只挑选一个学生来朗读，即一位学生的话语行为，符合”个体发言“特征，老师发起了一个开展”个体发言“课堂活动的指令；如果老师接下来挑选多个同学分别开展朗读活动，即多位学生轮流分享各自的朗读成果，符合”多人展示“特征，则老师发起一个开展”多人展示“课堂活动的指令。综合上述整段话的分析过程，由于不知道老师说完这句话之后，会要求学生具体执行哪个课堂活动的指令，所以，在老师说完这段话后，预测的课堂活动类别是“个体发言”或”多人展示“，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”接下来我们来做个角色扮演游戏。张三，你来扮演顾客，李四，你现来扮演店主，开始吧。“
    分析过程：在这段话语中，”接下来我们来做个角色扮演游戏“体现出老师发起了一个开展课堂活动的指令，这句话明确了课堂活动的目的是开展角色扮演活动。“张三，你来扮演顾客，李四，你现来扮演店主”进一步体现出老师挑选了两名学生来开展角色扮演活动，即通过角色扮演来分享他们对角色的理解。综合上述整段话的分析过程，这个课堂活动是由多个学生执行、且向全班同学分享他们对角色的理解，符合”多人展示“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“多人展示”。因为执行该课堂活动的指令刚刚发出，因此 进行状态为“开始”。

    老师话语：”天黑前上床睡觉都是我们习以为常的。为什么在作者看来却需要慰藉呢？请同学们联系第四自然段想一想，好，你继续说。”
    分析过程：在这段话语中，先分析第一个子文本段——”请同学们联系第四自然段想一想”，这句话体现出老师发起了一个开展课堂活动的指令。首先，”同学们“体现出该活动是由全体学生共同开展的，进一步地，”联系第四自然段想一想”体现出老师要求学生联系第四自然段进行思考活动，思考活动是个体的大脑活动，无法集体整齐划一地开展，只能由每一个学生独自完成。综合第一个子文本段的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主思考活动，符合“独立练习”的特征，因而，老师发起了一个要求学生开展“独立练习”的指令。
    接下来，再分析第二个子文本段——“好，你继续说”，这句话体现出老师发起了开展第二个课堂活动的指令，该学生被要求用话语来表达自己没有说完的内容。综合第二个子文本段的分析过程，这个课堂活动是由一个学生执行、且是一种话语行为，符合”个体发言“的特征，因而，第二个指令是开展”个体发言“的课堂活动的指令。
    由于整段话语中没有使用明显的时间词汇来区分这两个指令的执行顺序，所以，无法确定老师说完这段话后具体执行哪个指令。因此，预测的课堂活动类别是“独立练习”或“个体发言”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。
    分析过程：在这段话语中，先分析第一个子文本段——“那我们再来请用坐伤的方式来完成这道题，好，做商大家完成的非常快。”，其中，“请用坐伤的方式来完成”体现出老师发起了一个课堂活动的指令。首先“我们”一词体现出该课堂活动是全体学生共同参与的，”用坐伤的方式来完成这道题“体现出老师要求学生按照规定的方式来完成练习题，由于这是面向全体学生发出的指令，学生无法集体整齐划一的完成练习题，所以，只能每一个学生独自完成。综合第一个子文本段的分析过程，这个课堂活动需要全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，因而，老师发出了一个要求学生开展“独立练习”的指令。
    接下来，再分析第二个子文本段——“那我们现在小组分享一下你的结果，看一下大家完成的是否是一致的。”，其中，”那我们现在小组分享一下”体现出老师发起了开展一个课堂活动的指令，这里的“小组分享一下”表明学生需要在小组内部互相分享或以小组为单位向全班分享成果，由于这两种情况都有可能，只能根据后文的内容进行判断。”看一下大家完成的是否是一致的“这里的”大家“可能指代每一个小组内部的全部成员，也可能指代全班同学，所以还是无法明确判断”小组分享一下“是以哪种方式进行分享。综合第二个子文本段的分析过程，如果老师要求小组内部通过互相讨论的的方式来达到自主学习的目的，符合“小组学习”的特征，那么老师发出了一个要求学生开展“小组学习”的指令；如果老师要求学生以小组为单位向全班同学分享学习成果，则符合“多人展示”的特征，那么老师发出了一个要求学生开展“多人展示”的指令。
    在整段话语中，“做商大家完成的非常快”体现出老师对学生完成任务的速度表示认可，这意味着第一个课堂活动——”独立练习“，已经结束，此外，“那我们现在小组分享一下”中的“现在”一词体现出老师要求立即执行第二个课堂活动——“小组学习”或”多人展示“。所以，老师说完这段话后，预测的课堂活动类别是“小组学习”或”多人展示”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“我希望每个小组现在讨论一下刚才的实验结果，等一会儿向全班汇报。”
    分析过程：在这段话语中，先分析第一个子文本段——“我希望每个小组现在讨论一下刚才的实验结果“，其中，”每个小组现在讨论一下”体现出老师发起了一个开展课堂活动的指令，要求小组内部互相讨论实验结果。综合第一个子文本段的分析过程，这个课堂活动是老师要求小组内部通过互相讨论来达到自主学习的目的，符合“小组学习”的特征。因而，老师发出了一个要求学生开展“小组学习”的指令。
    接下来，再分析第二个子文本段——“等一会儿向全班汇报”，其中”向全班汇报”体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生在其他同学面前分享汇报内容，由于第二个子文本段没有直接体现出该课堂活动的执行主体，只能结合第一个子文本段来看，“每个小组”暗示了分享活动的执行主体是班级内每一个小组。综合第二个子文本段的分析过程，这个课堂活动是学生以小组为单位分享汇报成果，符合”多人展示“的特征。因而，第二个指令是开展”多人展示“的课堂活动的指令。
    在整段话语中，”等一会儿”这个词明显体现出第一个指令执行完成后，才执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前安排，不需要被立即执行，此外，“现在讨论一下”中的“现在”一词体现出老师要求立即执行第一个课堂活动——”小组学习“。所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“请你们四人一组，探讨一下《红楼梦》中主要人物的性格特点。”
    分析过程：在这段话语中，”请你们四人一组“体现出老师发起了一个开展课堂活动的指令。首先，“请你们四人一组”体现出老师要求学生以四人为一个小组开展课堂活动。进一步地，“探讨一下”体现出老师要求小组内部深入讨论《红楼梦》中主要人物的性格特点的活动。
    综合上述整段话的分析过程，这个课堂活动是老师要求小组内部通过相互讨论来达到自主学习的目的，符合“小组学习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“接下来，请大家自行解答第五页的练习题，不懂的地方可以标记出来。”
     分析过程：在这段话语中，”请大家自行解答第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
    首先，“请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“自行解答第五页的练习题”中的”自行解答“体现出该课堂活动是由每一个学生独自完成的。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：大家开始做一下第五页的练习题，小明你上来做一下。
     分析过程：在这段话语中，“大家开始做一下第五页的练习题”体现出老师发起了一个开展课堂活动的指令。
    首先，“大家开始做一下”中的“大家”体现出这个课堂活动是由全体学生共同参与的，“做一下”体现出老师要求学生完成第五页的练习题，由于这是面向全体学生发出的指令，学生无法集体整齐划一的完成练习题，所以，只能每一个学生独自完成。进一步地，“小明你上来做一下”，其中“你上来”体现出老师要求学生到讲台上或班级前面的位置，“小明”可能是一个学生的名字，这里的”你“指代”小明“这位学生。”做一下“体现出老师要求该学生执行一个课堂活动，结合上一句话“第五页的练习题”可知，要求学生执行的课堂活动是完成第五页的练习题，然而，这句话并没有表明该学生需要展示或解释答案给其他同学听，只是上来完成练习题，这意味着该学生只是不和其他同学一样在自己的座位上完成练习题，而是换个位置完成。
    综合上述整段话的分析过程，全体学生在同一时间内，不同位置各自独立完成练习题，这个课堂活动是由全体学生参与，且每个学生独立完成的自主练习活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“妈妈要到什么时候才要到夜里？那我饿了，饿了也得到太阳落下时才准吃。跟他一样。哎，你试着读一读，不用同学举手，自己读一读。“
    分析过程：在这段话语中，“你试着读一读”体现出老师发起了一个开展课堂活动的指令。
    首先，”你试着读一读“中的“你”可能指代一个学生，也可能指代全体学生。进一步地，“不用同学举手”中的“举手”是一种集体行为，如果班级只有一名同学，则不涉及“举手"这种行为，所以，“不用同学举手”中的“同学”指代全体同学，这也就意味着，上一句的”你试着读一读“中的”你”指代的也是全体同学。然后，“自己读一读”中的“自己”体现出老师要求每一个学生独自朗读。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。 所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“接下来我们进入课文，学习《春节的习俗》。请大家先看第一段，读一读，出声朗读，并反复诵读第三至第七自然段，边读边想象文字的场景。时间两分钟，开始。”
    分析过程：在这段话语中，“请大家先看第一段”体现出老师发起了一个开展课堂活动的指令。
    首先，”请大家”体现出这个课堂活动是由全体学生共同参与的。进一步地，“边读边想象文字的场景”体现出老师要求学生一边朗读一边想象文字的场景，因为想象属于个体的大脑活动，无法集体整齐划一地开展，所以，只能由学生独自完成。 
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主阅读活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，同学们，刚才我们讨论的那个场景，作者并没有对它的具体细节进行描绘。同学们，你们能不能运用你的想象力，详细描述一下这场大火是如何吞噬了整个森林的？你想象一下。“
    分析过程：在这段话语中，“运用你的想象力”体现出老师发起了一个课堂活动的指令。
    首先，”同学们“体现出这个课堂活动是由全体学生共同参与的。接下来地，”运用你的想象力“和“描述一下”体现出老师要求学生利用想象力深入描述一个场景，由于想象活动是个体的大脑思维活动，无法集体整齐划一地开展，所以，只能由每一个学生独自完成。“你的想象力”中的“你”结合前面的“你们能不能”可知，这里的“你”指代的还是全体学生。进一步地，“你想象一下”老师再次提醒学生想象大火如何吞噬整个森林的情景，这里的“你”结合前面的“同学们”依然指代全体学生。因而，整个课堂活动都是由全体学生共同参与的。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主想象活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“自学六七自然段默读自学提示，时间，七分钟，小组之间议一议，开始。”
    分析过程：在这段话语中，先分析第一个子文本段——”自学六七自然段默读自学提示，时间，七分钟“，其中”自学六七自然段”体现出老师发起了开展一个课堂活动的指令，首先，“自学六七自然段”中的”自学“体现出老师要求学生独自按照自学提示来学习六七自然段。进一步地，”时间，七分钟“体现出这个自学活动要在规定的时间范围内完成，这样的时间规定通常情况下是为了方便老师对学生的统一管理，所以这个自学活动的执行主体是全体学生。综合第一个子文本段的分析过程，这个课堂活动由全体学生参与、且每一个学生独自完成的自学活动，符合“独立练习”的特征，因此，老师发出了一个要求学生开展”独立练习“课堂活动的指令。
    接下来，再分析第二个子文本段——“小组之间议一议”，体现出老师发起了第二个课堂活动的指令，该指令体现出老师要求学生以小组的形式开展内部讨论活动。综合第二个子文本段的分析过程，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，因而，第二个指令是开展”小组学习“的课堂活动的指令。
    由于这段话中，“时间，七分钟“这个短语明显体现出第一个指令开展七分钟后，才开始执行第二个指令，这意味着第二个指令是老师对未来课堂活动的提前预告或提前安排，不需要被立即执行。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”在干涸的河床上，一只乌鸦辛苦地寻找水源。它终于在一个石头缝中发现了一点水。来，让我们一起来感受一下乌鸦找水的艰辛。”
    分析过程：在这段话语中，“让我们一起来感受一下乌鸦找水的艰辛。“体现出老师发起了一个开展课堂活动的指令。
    首先，“让我们一起来”中的”我们“一词体现出这个课堂活动是老师邀请全体学生共同参与的，其中“一起”有两种可能的理解：第一种理解所有学生整齐划一地进行同一个活动；第二种理解是每位学生各自独立参与，但都在做同样的事情。进一步地，“感受一下”体现出老师邀请学生参与感受活动，因为感受活动是个体的心理体验活动，无法集体整齐划一地开展，只能由每一个学生独自完成，因而，这段话语中的”一起“是第二种理解。 
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主感受活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“请我们像他一样，用一句话，把这个故事哎就讲出来了，来，我们一起来试试看。”
    分析过程：在这段话中，“我们一起来试试看”体现出老师发起了开展一个课堂活动的指令。
    首先，“我们一起来试试看“中”的”我们“体现出这个课堂活动是老师邀请全体学生共同参与的。“一起”这个词有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的事情。进一步地，‘试试看“体现出老师想要学生尝试做一件事情，结合上文“用一句话，把这个故事哎就讲出来了“可知，老师想要学生尝试的是用一句话将故事复述出来，因为复述活动是个体经过大脑思考后才能得出结论的活动，这种思考活动很难集体整齐划一地开展，只能由每一个学生独自完成，因而，这里的“一起”是第二种理解 。综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主复述活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“现在我会念一段诗句，大家请认真听写下来。”
    分析过程：在这段话语中，”请认真听写“体现出老师发起了开展一个课堂活动的指令。
    首先，”大家“体现出这个课堂活动是由全体学生共同参与的。进一步地，”认真听写“中的”听写“一词很明显体现出老师让学生开展听写活动。
    综合上述子文本段的分析过程，这个课堂活动是老师邀请全体学生参与的听写活动，符合”学生听写“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生听写”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，这课要请男孩子来做杜鹃，好不好？把这个话来读一读啊，正当桃树成活的时候，预备起，正当桃树成活的时候”
    分析过程：在这段话语中，“这课要请男孩子来做杜鹃“体现出老师发起了一个开展课堂活动的指令。
    首先，”这课要请男孩子来做杜鹃”中的”男孩子”，因为没有“一个”这类量词来修饰“男孩子“这个词，所以，这里的”男孩子“指代全体男同学这个群体。“来做杜鹃”体现了老师要求男同学这个群体扮演杜鹃这个角色，但并未明确阐述扮演杜鹃的具体目的或随后的活动内容。接下来地，“把这个话来读一读”中的“读一读”体现出老师让男同学扮演杜鹃的目的是通过这种角色扮演方式来朗诵杜鹃的台词，是一种出声朗读活动。然后，“预备起”这个词体现出这个出声朗读活动需要学生整齐划一地开展。
    综合上述整段话语的分析过程，这个出声朗读活动是老师邀请全体男同学这个群体参与、且整齐划一地开展，符合“学生齐读”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“我来读前面的你们来读第二大段，我一向讨厌母鸡。”
    分析过程：在这段话语中，”我来读前面的你们来读第二大段“体现出老师发起了一个开展课堂活动的指令。
    首先，“我来读前面的，你们来读第二大段”明确分配了朗读任务，其中“我”指老师自身，而“你们”则代表在场的全体学生。这样的用词表明了一种明确的分工：老师首先领读文章的开头部分，随后由学生集体跟读后续部分。进一步地，这种老师领读，全体学生跟读的课堂活动，通常情况下，老师要求学生整齐划一地开展。
    综合上述整段话的分析过程，这个课堂活动是老师领读，全体学生整齐划一地跟读，符合”学生齐读“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐读”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”那让我们合作来读一读老师来读看到的部分，你们来读想象到的部分。最后一句话我们一起来读，好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”
    分析过程：在这段话语中，“最后一句话我们一起来读“体现出老师发起了一个开展课堂活动的指令。
    首先，“最后一句话我们一起来读”中的“我们一起来”体现出这个课堂活动是老师邀请学生集体参与的。“读”字体现出老师邀请学生参与出声朗读活动。由于后面的”好不好？好好了吗？好了，嗯，你会从那小玻璃上面的一粒星一朵云，”信息不明确，通过这句话只知道朗读的内容，无法进一步推测朗读活动的具体执行形方式：学生集体整齐划一地朗读还是每一位学生独自朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
    综合上述整段话的分析过程，由于没有明确的信息来判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令，所以，老师说完这段话后，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：”同学们，大声的朗读课文。“
    分析过程：在这段话语中，”大声的朗读课文“体现出老师发起了一个课堂活动的指令。
    首先，”同学们“体现出老师邀请学生集体参与课堂活动。接下来地，“大声的朗读课文”明确了活动的具体内容，即学生们需要在朗读课文时保持较大的音量，这句话没有进一步明确朗读活动的具体执行形式：学生集体整齐划一地出声朗读还是每一位学生独自出声朗读。如果是要求学生集体整齐划一地出声朗读，符合”学生齐读“的特征，那么老师发出了一个要求学生开展“学生齐读”的指令；如果是要求全体学生参与、且每一个学生独自完成的自主出声朗读活动，则符合”独立练习“的特征，那么老师发出了一个要求学生开展“独立练习”的指令。
    综合上述整段话的分析过程，由于没有明确的信息判别这段话后到底是让学生执行”学生齐读“的指令，还是让学生执行”独立练习“的指令。因此，预测的课堂活动类别是“学生齐读”或“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“他跪在地上旁边呢有个人拿着拐杖在打他的小屁股，对不对？就是叫他知错就改对的啊，所以，这个改字我们一起来写一写，来，小手笔伸出来一横折，预备起，一横折，二横三十五，“
    分析过程：在这段话语中，”这个改字我们一起来写一写“体现出老师发起了一个开展课堂活动的指令。
    首先，“我们一起来写一写”中的”我们“体现出这个课堂活动是老师邀请学生集体参与的。“一起”有两种可能的理解：第一种是所有学生整齐划一地进行同一个活动；第二种是每位学生各自独立参与，但都在做同样的行为。”写一写“体现出老师要求学生开展书写活动。进一步地，"预备起"这个词体现出这个书写活动要求学生整齐划一地开展，因而，这里的”一起“是第一种理解。 
    综合上述整段话的分析过程，这个书写活动是老师邀请学生集体参与、且整齐划一地开展，符合“学生齐写”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“学生齐写”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“让这个画面更怎么样了，做到了更有活力了。很好，你们都感受到了燕子加入了这幅图中，为这幅图增添了很多生趣。哎，现在我们开始闯第三关，“
    分析过程：在这段话语中，“我们开始闯第三关”体现出老师发出了一个课堂活动的指令。
    首先，“我们”这一词语明确体现出这个课堂活动是由全体学生集体参与的，而不是针对个别学生的指令。进一步地，“闯第三关”体现出老师要求学生参与一个闯关活动，根据这句话只能知道学生要参与一个课堂活动，但没有明确说明该课堂活动的内容细节以及参与形式。
    综合上述整段话的分析过程，这个课堂活动是由全体学生共同参与、且活动内容和参与形式都不明确，学生需要等待进一步的指导和安排，符合”集体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“集体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：课后作业做了没有？在二十九页，翻到二十九页。如果你是用网课的资料做的，你就把网课打印的资料拿出来。如果没有的话，你翻到课本二十九页。
    分析过程：在这段话语中，”把网课打印的资料拿出来”体现出老师发起了一个开展课堂活动的指令。
    首先，“如果你是用网课的资料做的“体现出班级内有一部分同学是用网课资料完成的课后作业，紧接着，“你就把网课打印的资料拿出来”中的”你“指代使用网课资料的这部分同学，“拿出来”只能体现出老师让这部分学生把资料拿出来，但老师未明确说明拿出资料后的具体操作。进一步地，”如果没有的话”结合上一句”如果你是用网课的资料“，体现出班级内有一部分同学，没有使用网课资料完成课后作业，紧接着，“你翻到课本二十九页”中的“你”指代没有使用网课资料的学生群体，“翻到”体现出没有使用网课资料的这部分同学翻书到二十九页，可以推测出这部分学生的课后作业写在了二十九页，同样，这里也未说明翻阅后需进行的具体活动。
    综合上述整段话语的分析过程，通过预设两种情况，老师确保了每个学生，无论他们以什么方式完成课后作业，都被纳入到这次课堂活动中，因而，说明这个课堂活动是由全体学生共同参与、且活动内容细节和参与形式都不明确，学生需要等待进一步的指导和安排，符合”集体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“集体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，最后这位同学来尝试一下， ”
    分析过程：在这段话语中，“最后这位同学来尝试一下”体现出老师发起了一个课堂活动的指令。
    首先，“最后这位同学”体现出老师邀请一名特定的学生参与课堂活动。接下来地，“来尝试一下”体现出老师让该学生试着做某件事情，但没有明确说明具体要做什么以及执行的具体形式也不清楚。
    综合上述整段话的分析过程，说明这是针对一名特定学生的指令，且指令内容和执行形式都不明确，学生需要等待进一步的指导和安排，符合”个体未知“的特征。所以，在老师说完这段话后，预测的课堂活动类别是“个体未知”。因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

    老师话语：“好，你是通过圈一圈的方法来学习，请坐张鹤“
    分析过程：在这段话语中，”你是通过圈一圈的方法来学习“这句话是老师对学生学习方法的概括，其中的”好“字是老师对学生学习方法的肯定。”请坐张鹤”中的”张鹤“可能是一位学生的名字，体现出学生已经回答完毕，老师请张鹤这位同学坐下。综合上述整段话分析过程，这段话只是在确认学生的回答，并没有明确的开展课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    老师话语：“明天，我们有一个小组展示，每个小组需要准备一个关于环境保护的短片。”
    分析过程：在这段话语中，“明天，我们有一个小组展示”体现出明天将会有一个小组展示的活动，虽然，这句胡体现出老师要求学生以小组的形式分享自己的成果，符合”多人展示“的特征，但是，“明天”一词明确指出了活动发生的时间是明天，而不是在今天这个课堂上，意味着这是老师对未来课堂活动的提前预告或安排，而不需要学生立即执行。
    综合上述整段话的分析过程，这段话语的本质是布置了一项课后作业，需要学生们在明天的课堂上进行展示，而非一个需要立即执行的课堂活动的指令。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    老师话语：”好，你们再想想我去听别人别的组啊。“
    分析过程：在这段话语中，“我去听别人别的组”体现出老师要离开当前这个小组，接着去别的小组那里，这表明老师正在对每个小组进行逐一辅导，此外，这也表明每个小组成员正在积极参与小组讨论活动。其次，“你们再想想”中的“你们”指代当前小组的全体成员，而“再想想”则表示老师希望当前小组继续深入思考和讨论问题，这进一步支持了小组讨论的活动正在进行中的观点。
    综合上述整段话的分析过程，班级被划分为多个小组，每个小组均在积极地进行讨论和交流。老师在完成对一个小组的指导后，便鼓励学生继续思考和讨论，随后前往下一个小组进行辅导。因此，这个课堂活动是老师要求小组之间通过互相讨论来达到自主学习的目的，符合“小组学习”的特征，所以，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为每个小组正在积极地参与讨论，而老师则在各小组间巡视，因此进行状态为“进行中”。

    老师话语：“有的同学读完了，已经开始在写任务单了。”
    分析过程：在这段话语中，“有的同学读完了，已经开始在写任务单了“说明了一部分学生完成了阅读任务，并且已经在写任务单，这体现了班级内学生的行为已经出现分化，即不是所有学生都同时完成阅读或写作，从而说明“阅读”和“写任务单”两个任务被全体学生共同承担，但每一个学生完成这两个任务的时长各不相同。进一步地，由于每一个学生完成这两个任务的具体时长不同，所以，这表明每一个学生都在独自执行任务，没有其他同学的协助。
    综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主练习活动，符合“独立练习”的特征，所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为这段话老师在提醒学生课堂活动的进展状态，意味着这是一个正在进行中的课堂活动，因此进行状态为“进行中”。

    老师话语：”所以它把水的质量设为了a克，非常好，来，我们先请坐来同学们，大家看他根据刚才的这个情境呢设出了水的质量之后，得到了两个代数式。这两个代数式是我们熟悉的整式吗，”
    分析过程：在这段话中，"我们先请坐来同学们，大家看他根据刚才的这个情境呢设出了水的质量之后，得到了两个代数式" 显示出该学生已完成回答，并被老师指引坐下。老师使用 "大家来看" 的表达方式来引导全班同学观察和思考刚才那位学生得到的两个代数式，接下来的询问 "这两个代数式是我们熟悉的整式吗" 旨在激发学生的思考和讨论，检测他们对代数式概念的理解。
    综合上述整段话的分析过程，这整段话语主要是老师的引导和询问，没有发起任何课堂活动。老师的语言主要是为了引导学生思考并对刚才的回答进行验证，而非开展新的教学活动。所以，在老师说完这段话后，无法明确识别出任何预设的课堂活动类别。

    待分析文本如下：
    """

# 定义处理函数的功能：该函数接收一个DataFrame，逐行处理其内容，通过API调用生成相应的输出，并将输出添加到DataFrame中
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # 非流式处理：逐行处理DataFrame中的文本
    for i in tqdm(range(len(df))):
        try:
            # 调用API生成结果
            completion = client.chat.completions.create(
                model="ep-20240709071356-5dlnm",  # 指定模型
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": df.loc[i, 'text']},
                ],
            )
            # 打印输出结果
            print("输出结果", completion.choices[0].message.content)
     
            # 将结果存入DataFrame的新列中
            df.loc[i, "doubao_output_new"] = completion.choices[0].message.content
            break
        except Exception as e:
            # 捕获异常并记录日志
            logging.error(f"Error processing row {i}: {e}")
            # 在DataFrame中记录错误
            df.loc[i, 'doubao_output_new'] = "error"

    return df

In [60]:
df = pd.read_excel("../../data/test_llm/高希娜.xlsx")

In [61]:
process_dataframe(df)

  0%|          | 0/42 [00:11<?, ?it/s]

输出结果 分析过程：在这段话语中，“那同学们”体现出老师发起了一个开展课堂活动的指令，结合前文老师描述的糖水情境，推测老师接下来可能会要求学生对这个情境进行思考、分析或回答相关问题。由于想象和思考活动是个体的大脑思维活动，无法集体整齐划一地开展，所以，只能由每一个学生独自完成。
综合上述整段话的分析过程，这个课堂活动是由全体学生参与、且每一个学生独自完成的自主思考活动，符合“独立练习”的特征。所以，在老师说完这段话后，预测的课堂活动类别是“独立练习”，因为执行该课堂活动的指令刚刚发出，因此进行状态为“开始”。

输出结果：
{
  "label": "独立练习",
  "status": "开始",
  "key_text": "那同学们"
} 


,S_T,E_T,text,label,doubao_output_new
0,2分18秒,2分37秒,我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，...,NaN,分析过程：在这段话语中，“那同学们”体现出老师发起了一个开展课堂活动的指令，结合前文老师描述...
1,2分45秒,3分1秒,哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，...,小组学习,nan
2,5分20秒,5分27秒,好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，,学生展示,nan
3,6分28秒,6分45秒,所以它把水的质量设为了a克，非常好，来，我们先请坐来同学们，大家看他根据刚才的这个情境呢设出...,独立练习,nan
4,6分53秒,7分0秒,那这样的代数式同学们会给它取个什么样的名字，哎，这就是我们今天的内容认识分式，,NaN,nan
5,7分14秒,9分11秒,请同学们独立完成学习卡中的四个情景问题，好，完成的同学请举手，好，那我需要同学来回答一下你的...,独立练习或学生展示,nan
6,9分17秒,9分24秒,x分之10有没有问题，非常好，请坐正确。好，第二题好，来这位同学后面这位。,学生展示,nan
7,9分32秒,9分38秒,哎，正确非常棒。好，来我们继续看一下。第三，小题来好，这位同学，,学生展示,nan
8,9分49秒,10分1秒,来看一下，来做对这道题的请举手呢？好，有没有同学有疑惑？好，没有哈，来，我们继续。第四小题好...,学生展示,nan
9,10分26秒,10分44秒,总共列出了七个这样的代数式。那这七个代数式是整式吗，不是，那他们有什么样的共同特征，哦，大家...,NaN,nan


In [63]:
df[0:5]

,S_T,E_T,text,label,doubao_output_new
0,2分18秒,2分37秒,我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，...,NaN,分析过程：在这段话语中，“那同学们”体现出老师发起了一个开展课堂活动的指令，结合前文老师描述...
1,2分45秒,3分1秒,哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，...,小组学习,nan
2,5分20秒,5分27秒,好同学们来拿个小组来分享一下自己的想法，来大胆的分享，好，来这位同学，,学生展示,nan
3,6分28秒,6分45秒,所以它把水的质量设为了a克，非常好，来，我们先请坐来同学们，大家看他根据刚才的这个情境呢设出...,独立练习,nan
4,6分53秒,7分0秒,那这样的代数式同学们会给它取个什么样的名字，哎，这就是我们今天的内容认识分式，,NaN,nan


In [1]:
def custom_split(content):
    # todo：这里的切割点可能会因为response的返回不同而产生变化，要写成一个函数把所有的切割点都包住。现在的切割点是“老师话语”
    split_points = ["老师话语"]  # 定义切割点
    result = []  # 初始化结果列表
    temp = content  # 临时存储内容
    for point in split_points:
        parts = temp.split(point)  # 根据切割点进行切割
        result.extend([part for part in parts if part])  # 添加非空部分到结果列表
    return result  # 返回结果列表

In [2]:
content = """老师话语：我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，
分析过程: 在这段话语中，并没有出现明显的特征短语或特征短句来表明老师发起了一个开展课堂活动的指令。整段话的重点在于教师阐述关于水溶解糖的现象及其引发的感受。这些表述并没有体现出任何学生即将要立即参与的具体课堂活动。因此，这段老师的话并没有发起一个明确的课堂活动指令。
\
{
  "label": "NA",
  "status": "NA",
  "key_text": "NA"
}
\
老师话语：哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，
分析过程: 在这段话语中，“先四人小组讨论”明确体现出老师发起了一个开展课堂活动的指令。
首先，“四人小组讨论”体现出这是通过小组方式进行讨论的活动，进一步的解释中，“借助学习卡上的问题，分享你的想法”说明了讨论的内容是围绕学习卡上的问题，并分享自己的看法。这一指令显示出活动的形式和内容均与“讨论”和“分享”相关，符合“小组学习”的特征。因此，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为开展该课堂活动的指令刚刚发出，因此进行状态为“开始”。

{
  "label": "小组学习",
  "status": "开始",
  "key_text": "先四人小组讨论一下借助学习卡上的问题，分享你的想法"
}"""

In [5]:
list1 = custom_split(content)
list1

['：我们常常有这样的体会。一杯水，我加入了10g糖嗯，水变甜了，我又加入了10g糖，充分搅拌汤，完全溶解，哎，水更甜了。那同学们，\n分析过程: 在这段话语中，并没有出现明显的特征短语或特征短句来表明老师发起了一个开展课堂活动的指令。整段话的重点在于教师阐述关于水溶解糖的现象及其引发的感受。这些表述并没有体现出任何学生即将要立即参与的具体课堂活动。因此，这段老师的话并没有发起一个明确的课堂活动指令。\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n',
 '：哦，有同学已经有想法了，还有的同学有点疑惑。那现在我们先四人小组讨论一下借助学习卡上的问题，分享你的想法，四人小组，\n分析过程: 在这段话语中，“先四人小组讨论”明确体现出老师发起了一个开展课堂活动的指令。\n首先，“四人小组讨论”体现出这是通过小组方式进行讨论的活动，进一步的解释中，“借助学习卡上的问题，分享你的想法”说明了讨论的内容是围绕学习卡上的问题，并分享自己的看法。这一指令显示出活动的形式和内容均与“讨论”和“分享”相关，符合“小组学习”的特征。因此，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为开展该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题，分享你的想法"\n}']

In [9]:
def extract_analysis(text):
    start_index = text.find("分析过程")
    print("start_index", start_index)
    if start_index != -1:
        return text[start_index:]
    return ""

In [10]:
[extract_analysis(item) for item in list1] 

start_index 65
start_index 60


['分析过程: 在这段话语中，并没有出现明显的特征短语或特征短句来表明老师发起了一个开展课堂活动的指令。整段话的重点在于教师阐述关于水溶解糖的现象及其引发的感受。这些表述并没有体现出任何学生即将要立即参与的具体课堂活动。因此，这段老师的话并没有发起一个明确的课堂活动指令。\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n',
 '分析过程: 在这段话语中，“先四人小组讨论”明确体现出老师发起了一个开展课堂活动的指令。\n首先，“四人小组讨论”体现出这是通过小组方式进行讨论的活动，进一步的解释中，“借助学习卡上的问题，分享你的想法”说明了讨论的内容是围绕学习卡上的问题，并分享自己的看法。这一指令显示出活动的形式和内容均与“讨论”和“分享”相关，符合“小组学习”的特征。因此，在老师说完这段话后，预测的课堂活动类别是“小组学习”，因为开展该课堂活动的指令刚刚发出，因此进行状态为“开始”。\n\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "先四人小组讨论一下借助学习卡上的问题，分享你的想法"\n}']

In [11]:
def custom_split(content):
    # 检查content是否以"###"开头
    if content.startswith("###"):
        split_points = ["###"]
    else:
        split_points = ["老师话语"]
    
    result = []  # 初始化结果列表
    temp = content  # 临时存储内容
    for point in split_points:
        parts = temp.split(point)  # 根据切割点进行切割
        result.extend([part for part in parts if part])  # 添加非空部分到结果列表
        # 移除第一个元素，因为它已经被用作切割点
        if result and not result[0]:
            result.pop(0)
    return result  # 返回结果列表

# 示例使用
content = "###标题###内容老师话语总结"
print(custom_split(content))


['标题', '内容老师话语总结']


In [5]:
text1 = """分析过程: 在这段话中，大部分学生参与了课堂活动并已经完成，这表明当前的活动已经接近尾声。在提醒学生完成状态的同时，老师还对学生的坐姿给予了关注，这通常表明课堂活动的结束，并转入下一个环节。此外，这段话没有明确指出下一个课堂活动指令。因此，可以判断当前课堂活动即将结束，而没有宣布具体的接续活动。

分析过程: 
```json
{
  "label": "独立练习",
  "status": "结束",
  "key_text": "大部分同学都已经完成了，对吗？对对，好坐姿"
}
```
```"""
text1.split("分析过程: \n```json")

['分析过程: 在这段话中，大部分学生参与了课堂活动并已经完成，这表明当前的活动已经接近尾声。在提醒学生完成状态的同时，老师还对学生的坐姿给予了关注，这通常表明课堂活动的结束，并转入下一个环节。此外，这段话没有明确指出下一个课堂活动指令。因此，可以判断当前课堂活动即将结束，而没有宣布具体的接续活动。\n\n',
 '\n{\n  "label": "独立练习",\n  "status": "结束",\n  "key_text": "大部分同学都已经完成了，对吗？对对，好坐姿"\n}\n```\n```']

In [29]:
anaysis_processes_1= ['分析过程：在这段话语中，没有明确的课堂活动指令，只有指出要在课文上作批注。并没有清楚说明学生需要进行何种类型的活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，没有明确的课堂活动指令，仅是提出对原文进行概括和分析的要求，没有清楚说明学生需要进行何种类型的活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“第一个问题解决了没有？”是对学生进度的询问，“分层”没有明确指令，只是提到分层，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“我就请同学来回答啊”体现出老师发起了一个开展课堂活动的指令，邀请学生进行发言活动。进一步地，“请同学来回答”表明这是由个体学生进行的发言活动。所以，预测的课堂活动类别是“个体发言”。课堂活动的进行状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "我就请同学来回答啊"\n}\n```\n\n', '分析过程：在这段话语中，“请你来讲讲你的想法”体现出老师发起了一个开展课堂活动的指令，邀请一个学生展开发言活动。因此，预测的课堂活动类别是“个体发言”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "请你来讲讲你的想法"\n}\n```\n\n', '分析过程：在这段话语中，“你当第一层是吧，第一层，没有吗？”是对个别学生的提问，“第二层呢”也是对学生的继续提问，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“那你告诉大家”体现出老师发起了一个开展课堂活动的指令，邀请学生进行发言活动。因此预测的课堂活动类别是“个体发言”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "那你告诉大家"\n}\n```\n\n', '分析过程：在这段话语中，“第二层是分析分喜，他们是最可爱的人分析哦，我们的份上写了哪些事件，”是对不同层次进行分析和总结，没有明确发出新的指令，因此，不足以判断具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“还有一件事情呢”是对某个学生提示和引导，并没有明确的课堂活动指令，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“先把掌声送给乐门”体现出老师对学生的赞许和鼓励，但并无具体的课堂活动指令，没有体现出明确的学生需要执行的活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“你说你的想法”体现出老师发起了一个开展课堂活动的指令，邀请学生发言。因此预测的课堂活动类别是“个体发言”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "你说你的想法"\n}\n```\n\n', '分析过程：在这段话语中，“呃首先还是我，我其实觉得我可以”体现得出发言者是老师自己，而不是发起了新的课堂活动指令。这段话语没有明确的课堂活动指令，所以无法判断具体的活动内容。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，全段话语只是对段落的分层进行解释和描述，并没有明确的课堂活动指令，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n', '分析过程：在这段话语中，“现在交流三分钟的时间，小组合作交流开”体现出老师发起了一个开展课堂活动的指令，要求学生以小组的形式开展讨论活动。因此预测的课堂活动类别是“小组学习”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "小组学习",\n  "status": "开始",\n  "key_text": "现在交流三分钟的时间，小组合作交流开"\n}\n```\n\n', '分析过程：在这段话语中，“有予你们的小组来到上面”体现出老师发起了一个开展课堂活动的指令，要求学生以小组的形式展示分析结果。从指令中可以看出，学生一部分负责找原句，另一部分负责写在黑板上。因此，预测的课堂活动类别是“多人展示”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "多人展示",\n  "status": "开始",\n  "key_text": "有予你们的小组来到上面"\n}\n```\n\n', '分析过程：在这段话语中，这只是对之前老师发言内容的回顾和提问，没有明确的课堂活动指令，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```']


In [30]:
anaysis_processes_2 =  ['分析过程：在这段话语中，描述了某人对三个词的提炼，“这不是底老师讲的内容吧”以及“为什么可爱很吃苦，贡献多要的少，不思无私。”这些内容主要是对某人的提炼进行评价和分析，并未包含任何具体课堂活动的指令。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语没有包含任何指令性内容，仅仅是结束了某个稍早之前的讨论，并表示感谢和形成新话题的过渡。而“这这小第第七小组第”未完整表达意思。所以无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，老师是在讨论“无私奉献”这个词语及其意义，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，“卢德浩请你补充”体现出老师发起了一个开展课堂活动的指令，邀请一位名叫“卢德浩”的同学靠话语表达他的见解。这属于要求个体发言，因此，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因该活动刚刚发起，故进行状态为“开始”。\n\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "卢德浩请你补充"\n}\n\n', '分析过程：在这段话语中，“还有要补充的吗？”体现出老师希望进一步获取学生的发言，但没有具体指向某位学生，也没有发起具体活动指令。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是对“中国人的热情”的评论，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是对“热情”这一话题进行进一步探讨，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是在解释作者的国籍以及志愿军对作者的态度，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，“行政，刚刚你要补充，请你继续啊”体现出老师发起了一个开展课堂活动的指令，邀请一位名为“行政”的学生继续他的发言。这属于要求个体发言，因此，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因该活动刚刚发起，故进行状态为“开始”。\n\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "行政，刚刚你要补充，请你继续啊"\n}\n\n', '分析过程：在这段话语中，“坐下”体现出老师对之前个体发言已经结束的指令。而“我是希望你们能够找到这些具体的语言，能够概括出这些方面来啊。”并没有具体的指令，仅是对课堂活动的期待和要求。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，虽然出现了有关修辞手法和表达方式的内容，没有具体指令学生进行某种课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，老师在解决某个问题，即解释表达方式，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是关于“记叙文”的解释和提问，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语只是对上一段的补充解释，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语解释了描写的不同角度及方式，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，老师仅在解释描写方法，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}']


In [31]:
anaysis_processes_3=['分析过程：在这段话语中，描述了某人对三个词的提炼，“这不是底老师讲的内容吧”以及“为什么可爱很吃苦，贡献多要的少，不思无私。”这些内容主要是对某人的提炼进行评价和分析，并未包含任何具体课堂活动的指令。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语没有包含任何指令性内容，仅仅是结束了某个稍早之前的讨论，并表示感谢和形成新话题的过渡。而“这这小第第七小组第”未完整表达意思。所以无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，老师是在讨论“无私奉献”这个词语及其意义，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，“卢德浩请你补充”体现出老师发起了一个开展课堂活动的指令，邀请一位名叫“卢德浩”的同学靠话语表达他的见解。这属于要求个体发言，因此，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因该活动刚刚发起，故进行状态为“开始”。\n\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "卢德浩请你补充"\n}\n\n', '分析过程：在这段话语中，“还有要补充的吗？”体现出老师希望进一步获取学生的发言，但没有具体指向某位学生，也没有发起具体活动指令。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是对“中国人的热情”的评论，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是对“热情”这一话题进行进一步探讨，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是在解释作者的国籍以及志愿军对作者的态度，并没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，“行政，刚刚你要补充，请你继续啊”体现出老师发起了一个开展课堂活动的指令，邀请一位名为“行政”的学生继续他的发言。这属于要求个体发言，因此，在老师说完这段话后，预测的课堂活动类别是“个体发言”，因该活动刚刚发起，故进行状态为“开始”。\n\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "行政，刚刚你要补充，请你继续啊"\n}\n\n', '分析过程：在这段话语中，“坐下”体现出老师对之前个体发言已经结束的指令。而“我是希望你们能够找到这些具体的语言，能够概括出这些方面来啊。”并没有具体的指令，仅是对课堂活动的期待和要求。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，虽然出现了有关修辞手法和表达方式的内容，没有具体指令学生进行某种课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语中，老师在解决某个问题，即解释表达方式，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语是关于“记叙文”的解释和提问，没有布置具体的课堂活动。因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语只是对上一段的补充解释，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：这段话语解释了描写的不同角度及方式，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n\n', '分析过程：在这段话语中，老师仅在解释描写方法，并没有布置具体的课堂活动，因此，无法识别出任何预设的课堂活动类别。\n\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}']


In [38]:
anaysis_processes_4 =  ['errro','分析过程：在这段话语中，“花两分钟的时间，先浏览全文，带着问题去浏览文”体现出老师发起了一个开展课堂活动的指令。首先，“先浏览全文，带着问题去浏览文”体现出这个课堂活动是由全体学生共同参与的，且每个学生独自浏览文章和思考问题，因此，符合“独立练习”的特征。接下来的话，“然后要要请学学老师师要来讲课啊，还是讲三道题。像昨天周艺兴和朗朗是吧？”中并未明确指出要进行的课堂活动，不过其含义为可能请特定学生或老师来讲解，但未能明确具体指令。\n\n综合上述整段话的分析过程，目前明确的是老师指示学生浏览全文并思考问题的“独立练习”指令，其它部分并没有明确具体的课堂活动指令。\n\n```json\n{\n  "label": "独立练习",\n  "status": "开始",\n  "key_text": "先浏览全文，带着问题去浏览文"\n}\n```']


In [39]:
list1 = []
list1.extend(anaysis_processes_1)
list1.extend(anaysis_processes_2)
list1.extend(anaysis_processes_3)
list1.extend(anaysis_processes_4)

In [40]:
list1 

['分析过程：在这段话语中，没有明确的课堂活动指令，只有指出要在课文上作批注。并没有清楚说明学生需要进行何种类型的活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n',
 '分析过程：在这段话语中，没有明确的课堂活动指令，仅是提出对原文进行概括和分析的要求，没有清楚说明学生需要进行何种类型的活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n',
 '分析过程：在这段话语中，“第一个问题解决了没有？”是对学生进度的询问，“分层”没有明确指令，只是提到分层，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_text": "NA"\n}\n```\n\n',
 '分析过程：在这段话语中，“我就请同学来回答啊”体现出老师发起了一个开展课堂活动的指令，邀请学生进行发言活动。进一步地，“请同学来回答”表明这是由个体学生进行的发言活动。所以，预测的课堂活动类别是“个体发言”。课堂活动的进行状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "我就请同学来回答啊"\n}\n```\n\n',
 '分析过程：在这段话语中，“请你来讲讲你的想法”体现出老师发起了一个开展课堂活动的指令，邀请一个学生展开发言活动。因此，预测的课堂活动类别是“个体发言”，因为执行该活动的指令刚刚发出，因此活动状态为“开始”。\n\n```json\n{\n  "label": "个体发言",\n  "status": "开始",\n  "key_text": "请你来讲讲你的想法"\n}\n```\n\n',
 '分析过程：在这段话语中，“你当第一层是吧，第一层，没有吗？”是对个别学生的提问，“第二层呢”也是对学生的继续提问，不足以判断接下来的具体活动。\n\n```json\n{\n  "label": "NA",\n  "status": "NA",\n  "key_

In [46]:
import pandas as pd
df = pd.read_excel("../../../data//3有gpt4o分析过程的数据/705_processed.xlsx")

In [47]:
df

,S_T,E_T,text,label
0,1分17秒,1分19秒,直接在课文上批注出来，,NaN
1,1分24秒,1分31秒,并概括义。你认为谁是最可爱的人结着原文分析，,NaN
2,4分59秒,5分1秒,第一个问题解决了没有？分层，,NaN
3,5分13秒,5分32秒,然后概括诚意五分钟的时间。这已经是我们第次次学篇文章章网课课期间，杰杰师师这篇文章的是吧？所...,NaN
4,5分37秒,5分54秒,请你来讲讲你的想法，你怎么分的，对，分层就分分段落分层次。呃，我是这样分的，一二三段呢，,NaN
5,6分1秒,6分35秒,你当第一层是吧，第一层，没有吗？一到三门第一层还有，第二层呢，,NaN
6,6分49秒,7分1秒,四到十三是吧，你也是这样分的，四到十三，那后面都是十四到五十五是吧，这是你的分法，对不对？那...,NaN
7,7分31秒,7分46秒,为什么他们是最可爱的人，总写他们是最可爱的人是吧，哦，是不是？那第第二层呢？第二层是分析分喜...,NaN
8,7分50秒,8分4秒,后面是古风二次战役中的一个松古风，后面的就是后面就是马马玉霞从东幼孩子。对嗯，还有一件事情呢，,NaN
9,8分37秒,8分52秒,写了分写了这三个他们最可爱的事件。嗯，那第三层呢总还是总写还是总写，所以是总总分总啊，这是这...,NaN


In [48]:
df['gpt4o-output'] = list1 


In [50]:
df

,S_T,E_T,text,label,gpt4o-output
0,1分17秒,1分19秒,直接在课文上批注出来，,NaN,分析过程：在这段话语中，没有明确的课堂活动指令，只有指出要在课文上作批注。并没有清楚说明学生...
1,1分24秒,1分31秒,并概括义。你认为谁是最可爱的人结着原文分析，,NaN,分析过程：在这段话语中，没有明确的课堂活动指令，仅是提出对原文进行概括和分析的要求，没有清楚...
2,4分59秒,5分1秒,第一个问题解决了没有？分层，,NaN,分析过程：在这段话语中，“第一个问题解决了没有？”是对学生进度的询问，“分层”没有明确指令，...
3,5分13秒,5分32秒,然后概括诚意五分钟的时间。这已经是我们第次次学篇文章章网课课期间，杰杰师师这篇文章的是吧？所...,NaN,分析过程：在这段话语中，“我就请同学来回答啊”体现出老师发起了一个开展课堂活动的指令，邀请学...
4,5分37秒,5分54秒,请你来讲讲你的想法，你怎么分的，对，分层就分分段落分层次。呃，我是这样分的，一二三段呢，,NaN,分析过程：在这段话语中，“请你来讲讲你的想法”体现出老师发起了一个开展课堂活动的指令，邀请一...
5,6分1秒,6分35秒,你当第一层是吧，第一层，没有吗？一到三门第一层还有，第二层呢，,NaN,分析过程：在这段话语中，“你当第一层是吧，第一层，没有吗？”是对个别学生的提问，“第二层呢”...
6,6分49秒,7分1秒,四到十三是吧，你也是这样分的，四到十三，那后面都是十四到五十五是吧，这是你的分法，对不对？那...,NaN,分析过程：在这段话语中，“那你告诉大家”体现出老师发起了一个开展课堂活动的指令，邀请学生进行...
7,7分31秒,7分46秒,为什么他们是最可爱的人，总写他们是最可爱的人是吧，哦，是不是？那第第二层呢？第二层是分析分喜...,NaN,分析过程：在这段话语中，“第二层是分析分喜，他们是最可爱的人分析哦，我们的份上写了哪些事件，...
8,7分50秒,8分4秒,后面是古风二次战役中的一个松古风，后面的就是后面就是马马玉霞从东幼孩子。对嗯，还有一件事情呢，,NaN,分析过程：在这段话语中，“还有一件事情呢”是对某个学生提示和引导，并没有明确的课堂活动指令，...
9,8分37秒,8分52秒,写了分写了这三个他们最可爱的事件。嗯，那第三层呢总还是总写还是总写，所以是总总分总啊，这是这...,NaN,分析过程：在这段话语中，“先把掌声送给乐门”体现出老师对学生的赞许和鼓励，但并无具体的课堂活...


In [51]:
df.to_excel("../../../data//3有gpt4o分析过程的数据/705_processed.xlsx")

In [1]:
# 合并跑出来的数据 

In [10]:
import os 
import pandas as pd
os.listdir("data")

['705_processed.xlsx',
 '726_processed.xlsx',
 'labazhou_processed.xlsx',
 'renzhi_processed.xlsx',
 'tianchuang2_processed.xlsx',
 'wuhan1_asr_voice_processed.xlsx',
 'wuhan2_asr_voice_processed.xlsx',
 'yanzi_processed.xlsx']

In [11]:
list_df = []
for item in os.listdir("data"):
    df = pd.read_excel("data/"+item)
    list_df.append(df)

In [12]:
import pandas as pd
# 定义文件名列表
file_names =  os.listdir("data")
# 定义统一的列顺序
						

common_columns = ['S_T', 'E_T', 'text', 'label','qwen_long_output','gpt4o_output','doubao_output_new']  # 用你实际的列名替换这里
# 读取并合并文件
combined_df = pd.DataFrame()
for file in file_names:
   df = pd.read_excel("data/"+file)  # 读取 Excel 文件
   # 重新排列列的顺序
   df = df[common_columns]
   # 将当前文件的数据追加到合并后的 DataFrame
   combined_df = pd.concat([combined_df, df], ignore_index=True)
# 将合并后的 DataFrame 写入新的 Excel 文件
# combined_df.to_excel('combined_file.xlsx', index=False)

In [14]:
combined_df = combined_df[~combined_df['gpt4o_output'].str.contains("error")]
combined_df.to_excel('combined_file.xlsx', index=False)

In [15]:
combined_df

,S_T,E_T,text,label,qwen_long_output,gpt4o_output,doubao_output_new
0,1分17秒,1分19秒,直接在课文上批注出来，,NaN,分析过程：在这段话语中，“直接在课文上批注出来”体现出老师发起了一个开展课堂活动的指令，要求...,分析过程：在这段话语中，“直接在课文上批注出来”体现出老师发起了一个开展课堂活动的指令。首先...,分析过程：在这段话语中，“直接在课文上批注出来”体现出老师发起了一个开展课堂活动的指令，要求...
1,1分24秒,1分31秒,并概括义。你认为谁是最可爱的人结着原文分析，,NaN,分析过程：在这段话语中，“并概括义。你认为谁是最可爱的人结着原文分析”体现出老师发起了一个开...,分析过程：在这段话语中，“并概括义”中，“概括”体现出老师发起了一个开展课堂活动的指令，让学...,分析过程：在这段话语中，“并概括义。你认为谁是最可爱的人结着原文分析”体现出老师发起了一个开...
2,4分59秒,5分1秒,第一个问题解决了没有？分层，,NaN,分析过程：在这段话语中，“第一个问题解决了没有？”体现出老师在询问学生是否解决了某个问题，而...,分析过程：在这段话语中，“第一个问题解决了没有？”体现出老师在询问学生是否解决了第一个问题；...,分析过程：在这段话语中，“第一个问题解决了没有？分层”体现出老师发起了一个询问课堂活动进度的...
3,5分13秒,5分32秒,然后概括诚意五分钟的时间。这已经是我们第次次学篇文章章网课课期间，杰杰师师这篇文章的是吧？所...,NaN,分析过程：在这段话语中，“然后概括诚意五分钟的时间”体现出老师发起了一个开展课堂活动的指令，...,"分析过程：在这段话语中，""我就请同学来回答啊，我们先不交流这个问题，就是看你的这个还是没你的...",分析过程：在这段话语中，“我就请同学来回答啊”体现出老师发起了一个开展课堂活动的指令，邀请一...
4,5分37秒,5分54秒,请你来讲讲你的想法，你怎么分的，对，分层就分分段落分层次。呃，我是这样分的，一二三段呢，,NaN,分析过程：在这段话语中，“请你来讲讲你的想法”体现出老师发起了一个开展课堂活动的指令，邀请一...,分析过程：在这段话语中，“请你来讲讲你的想法”体现出老师发起了一个开展课堂活动的指令，邀请一...,分析过程：在这段话语中，“请你来讲讲你的想法，你怎么分的”体现出老师发起了一个开展课堂活动的...
...,...,...,...,...,...,...,...
374,30分16秒,30分33秒,好，同学们，作者用这么优美的笔触来写小燕子来写春天，你读完了这篇课文，你有怎样的感受？你体会...,NaN,分析过程：在这段话语中，“你读完了这篇课文，你有怎样的感受？你体会到了作者的情感是怎样的”体...,分析过程：在这段话语中，“你读完了这篇课文，你有怎样的感受？”体现出老师发起了一个开展课堂活...,分析过程：在这段话语中，“好，同学们，作者用这么优美的笔触来写小燕子来写春天，你读完了这篇课...
375,31分36秒,31分53秒,好，那在这里啊，作者运用了这么多美好的事物和笔触，去描写这篇课文。那现在啊请你找一找文中优美...,NaN,分析过程：在这段话语中，“请你找一找文中优美生动的语句，把它抄写在你们的任务单上”体现出老师...,分析过程：在这段话语中，“那现在啊请你找一找文中优美生动的语句，把它抄写在你们的任务单上”体...,分析过程：在这段话语中，“那现在啊请你找一找文中优美生动的语句，把它抄写在你们的任务单上”体...
376,32分54秒,33分19秒,每一笔，提前完成的同学可以写下边的奖励任务，,NaN,分析过程：在这段话语中，“每一笔”可能指的是课程中的每一项任务或练习，而“提前完成的同学可以...,分析过程：在这段话语中，“提前完成的同学可以写下边的奖励任务”体现出老师发起了两个开展课堂活...,分析过程：在这段话语中，“每一笔，提前完成的同学可以写下边的奖励任务”体现出老师发起了一个开...
377,38分0秒,38分18秒,大家一起抬头来欣赏一下这几位同学的书写，再来评一评，感觉怎么样，最喜欢哪一个，或者说你想评一...,NaN,"分析过程：在这段话语中，""大家一起抬头来欣赏一下这几位同学的书写“体现出老师邀请全体学生参与...",分析过程：在这段话语中，第一步，先分析第一个子文本段——“大家一起抬头来欣赏一下这几位同学的...,分析过程：在这段话语中，“大家一起抬头来欣赏一下这几位同学的书写”体现出老师发起了一个开展课...
